In [1]:
from perses.app.relative_point_mutation_setup import PointMutationExecutor
import pickle

INFO:numexpr.utils:Note: detected 72 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 72 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:rdkit:Enabling RDKit 2021.03.4 jupyter extensions


In [2]:
from perses.utils.openeye import createOEMolFromSDF, extractPositionsFromOEMol
from perses.annihilation.relative import HybridTopologyFactory, RepartitionedHybridTopologyFactory
from perses.rjmc.topology_proposal import PointMutationEngine
from perses.rjmc.geometry import FFAllAngleGeometryEngine

import simtk.openmm as openmm
import simtk.openmm.app as app
import simtk.unit as unit
import numpy as np
from openmoltools import forcefield_generators
import mdtraj as md
from openmmtools.constants import kB
from perses.tests.utils import validate_endstate_energies
from openff.toolkit.topology import Molecule
from openmmforcefields.generators import SystemGenerator

ENERGY_THRESHOLD = 1e-2
temperature = 300 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
ring_amino_acids = ['TYR', 'PHE', 'TRP', 'PRO', 'HIS']

# Set up logger
import logging
_logger = logging.getLogger("setup")
_logger.setLevel(logging.INFO)



In [3]:
class PointMutationExecutor2(PointMutationExecutor):
    """
    Simple, stripped-down class to create a protein-ligand system and allow a mutation of a protein.
    this will allow support for the creation of _two_ relative free energy calculations:
        1. 'wildtype' - 'point mutant' complex hybrid.
        2. 'wildtype' - 'point mutant' protein hybrid (i.e. with ligand of interest unbound)
    Example (create full point mutation executor and run parallel tempering on both complex and apo phases):
        from pkg_resources import resource_filename
        protein_path = 'data/perses_jacs_systems/thrombin/Thrombin_protein.pdb'
        ligands_path = 'data/perses_jacs_systems/thrombin/Thrombin_ligands.sdf'
        protein_filename = resource_filename('openmmforcefields', protein_path)
        ligand_input = resource_filename('openmmforcefields', ligands_path)
        pm_delivery = PointMutationExecutor(protein_filename=protein_filename,
                                    mutation_chain_id='2',
                                    mutation_residue_id='198',
                                     proposed_residue='THR',
                                     phase='complex',
                                     conduct_endstate_validation=False,
                                     ligand_input=ligand_input,
                                     ligand_index=0,
                                     forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml'],
                                     barostat=openmm.MonteCarloBarostat(1.0 * unit.atmosphere, temperature, 50),
                                     forcefield_kwargs={'removeCMMotion': False, 'ewaldErrorTolerance': 1e-4, 'nonbondedMethod': app.PME, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus},
                                     small_molecule_forcefields='gaff-2.11')
        complex_htf = pm_delivery.get_complex_htf()
        apo_htf = pm_delivery.get_apo_htf()
        # Now we can build the hybrid repex samplers
        from perses.annihilation.lambda_protocol import LambdaProtocol
        from openmmtools.multistate import MultiStateReporter
        from perses.samplers.multistate import HybridRepexSampler
        from openmmtools import mcmc
        suffix = 'run'; selection = 'not water'; checkpoint_interval = 10; n_states = 11; n_cycles = 5000
        for htf in [complex_htf, apo_htf]:
            lambda_protocol = LambdaProtocol(functions='default')
            reporter_file = 'reporter.nc'
            reporter = MultiStateReporter(reporter_file, analysis_particle_indices = htf.hybrid_topology.select(selection), checkpoint_interval = checkpoint_interval)
            hss = HybridRepexSampler(mcmc_moves=mcmc.LangevinSplittingDynamicsMove(timestep= 4.0 * unit.femtoseconds,
                                                                                  collision_rate=5.0 / unit.picosecond,
                                                                                  n_steps=250,
                                                                                  reassign_velocities=False,
                                                                                  n_restart_attempts=20,
                                                                                  splitting="V R R R O R R R V",
                                                                                  constraint_tolerance=1e-06),
                                                                                  hybrid_factory=htf,
                                                                                  online_analysis_interval=10,
                                                                                  context_cache=cache.ContextCache(capacity=None, time_to_live=None))
            hss.setup(n_states=n_states, temperature=300*unit.kelvin, storage_file=reporter, lambda_protocol=lambda_protocol, endstates=False)
            hss.extend(n_cycles)
    """
    def __init__(self,
                 protein_filename,
                 mutation_chain_id,
                 mutation_residue_id,
                 proposed_residue,
                 phase='complex',
                 conduct_endstate_validation=True,
                 ligand_input=None,
                 ligand_index=0,
                 allow_undefined_stereo_sdf=False,
                 water_model='tip3p',
                 ionic_strength=0.15 * unit.molar,
                 forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml'],
                 barostat=openmm.MonteCarloBarostat(1.0 * unit.atmosphere, temperature, 50),
                 forcefield_kwargs={'removeCMMotion': False, 'ewaldErrorTolerance': 0.00025, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus},
                 periodic_forcefield_kwargs={'nonbondedMethod': app.PME},
                 nonperiodic_forcefield_kwargs=None,
                 small_molecule_forcefields='gaff-2.11',
                 complex_box_dimensions=None,
                 apo_box_dimensions=None,
                 flatten_torsions=False,
                 flatten_exceptions=False,
                 generate_unmodified_hybrid_topology_factory=True,
                 generate_rest_capable_hybrid_topology_factory=False,
                 **kwargs):
        """
        arguments
            protein_filename : str
                path to protein (to mutate); .pdb
            mutation_chain_id : str
                name of the chain to be mutated
            mutation_residue_id : str
                residue id to change
            proposed_residue : str
                three letter code of the residue to mutate to
            phase : str, default complex
                if phase == vacuum, then the complex will not be solvated with water; else, it will be solvated with tip3p
            conduct_endstate_validation : bool, default True
                whether to conduct an endstate validation of the HybridTopologyFactory. If using the RepartitionedHybridTopologyFactory,
                endstate validation cannot and will not be conducted.
            ligand_input : str, default None
                path to ligand of interest (i.e. small molecule or protein); .sdf or .pdb
            ligand_index : int, default 0
                which ligand to use
            allow_undefined_stereo_sdf : bool, default False
                whether to allow an SDF file to contain undefined stereocenters
            water_model : str, default 'tip3p'
                solvent model to use for solvation
            ionic_strength : float * unit.molar, default 0.15 * unit.molar
                the total concentration of ions (both positive and negative) to add using Modeller.
                This does not include ions that are added to neutralize the system.
                Note that only monovalent ions are currently supported.
            forcefield_files : list of str, default ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
                forcefield files for proteins and solvent
            barostat : openmm.MonteCarloBarostat, default openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
                barostat to use
            forcefield_kwargs : dict, default {'removeCMMotion': False, 'ewaldErrorTolerance': 1e-4, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus}
                forcefield kwargs for system parametrization
            periodic_forcefield_kwargs : dict, default {'nonbondedMethod': app.PME}
                periodic forcefield kwargs for system parametrization
            nonperiodic_forcefield_kwargs : dict, default None
                non-periodic forcefield kwargs for system parametrization
            small_molecule_forcefields : str, default 'gaff-2.11'
                the forcefield string for small molecule parametrization
            complex_box_dimensions : Vec3, default None
                define box dimensions of complex phase;
                if None, padding is 1nm
            apo_box_dimensions :  Vec3, default None
                define box dimensions of apo phase phase;
                if None, padding is 1nm
            flatten_torsions : bool, default False
                in the htf, flatten torsions involving unique new atoms at lambda = 0 and unique old atoms are lambda = 1
            flatten_exceptions : bool, default False
                in the htf, flatten exceptions involving unique new atoms at lambda = 0 and unique old atoms at lambda = 1
            generate_unmodified_hybrid_topology_factory : bool, default True
                whether to generate a vanilla HybridTopologyFactory
            generate_rest_capable_hybrid_topology_factory : bool, default False
                whether to generate a RepartitionedHybridTopologyFactory
        TODO : allow argument for spectator ligands besides the 'ligand_file'
        """
        from openeye import oechem

        # First thing to do is load the apo protein to mutate...
        protein_pdbfile = open(protein_filename, 'r')
        protein_pdb = app.PDBFile(protein_pdbfile)
        protein_pdbfile.close()
        protein_positions, protein_topology, protein_md_topology = protein_pdb.positions, protein_pdb.topology, md.Topology.from_openmm(protein_pdb.topology)
        protein_topology = protein_md_topology.to_openmm()
        protein_n_atoms = protein_md_topology.n_atoms

        # Load the ligand, if present
        molecules = []
        if ligand_input:
            if isinstance(ligand_input, str):
                if ligand_input.endswith('.sdf'): # small molecule
                        ligand_mol = createOEMolFromSDF(ligand_input, index=ligand_index, allow_undefined_stereo=allow_undefined_stereo_sdf)
                        molecules.append(Molecule.from_openeye(ligand_mol, allow_undefined_stereo=False))
                        ligand_positions, ligand_topology = extractPositionsFromOEMol(ligand_mol),  forcefield_generators.generateTopologyFromOEMol(ligand_mol)
                        ligand_md_topology = md.Topology.from_openmm(ligand_topology)
                        ligand_n_atoms = ligand_md_topology.n_atoms

                if ligand_input.endswith('pdb'): # protein
                    ligand_pdbfile = open(ligand_input, 'r')
                    ligand_pdb = app.PDBFile(ligand_pdbfile)
                    ligand_pdbfile.close()
                    ligand_positions, ligand_topology, ligand_md_topology = ligand_pdb.positions, ligand_pdb.topology, md.Topology.from_openmm(
                        ligand_pdb.topology)
                    ligand_n_atoms = ligand_md_topology.n_atoms

            elif isinstance(ligand_input, oechem.OEMol): # oemol object
                molecules.append(Molecule.from_openeye(ligand_input, allow_undefined_stereo=False))
                ligand_positions, ligand_topology = extractPositionsFromOEMol(ligand_input),  forcefield_generators.generateTopologyFromOEMol(ligand_input)
                ligand_md_topology = md.Topology.from_openmm(ligand_topology)
                ligand_n_atoms = ligand_md_topology.n_atoms

            else:
                _logger.warning(f'ligand filetype not recognised. Please provide a path to a .pdb or .sdf file')
                return

            # Now create a complex
            complex_md_topology = protein_md_topology.join(ligand_md_topology)
            complex_topology = complex_md_topology.to_openmm()
            complex_positions = unit.Quantity(np.zeros([protein_n_atoms + ligand_n_atoms, 3]), unit=unit.nanometers)
            complex_positions[:protein_n_atoms, :] = protein_positions
            complex_positions[protein_n_atoms:, :] = ligand_positions

            # Convert positions back to openmm vec3 objects
            complex_positions_vec3 = []
            for position in complex_positions:
                complex_positions_vec3.append(openmm.Vec3(*position.value_in_unit_system(unit.md_unit_system)))
            complex_positions = unit.Quantity(value=complex_positions_vec3, unit=unit.nanometer)

        # Now for a system_generator
        self.system_generator = SystemGenerator(forcefields=forcefield_files,
                                                barostat=barostat,
                                                forcefield_kwargs=forcefield_kwargs,
                                                periodic_forcefield_kwargs=periodic_forcefield_kwargs,
                                                nonperiodic_forcefield_kwargs=nonperiodic_forcefield_kwargs,
                                                small_molecule_forcefield=small_molecule_forcefields,
                                                molecules=molecules,
                                                cache=None)

        # Solvate apo and complex...
        apo_input = list(self._solvate(protein_topology, protein_positions, water_model, phase, ionic_strength, apo_box_dimensions))
        inputs = [apo_input]
        if ligand_input:
            inputs.append(self._solvate(complex_topology, complex_positions, water_model, phase, ionic_strength, complex_box_dimensions))

        geometry_engine = FFAllAngleGeometryEngine(metadata=None,
                                                use_sterics=False,
                                                n_bond_divisions=100,
                                                n_angle_divisions=180,
                                                n_torsion_divisions=360,
                                                verbose=True,
                                                storage=None,
                                                bond_softening_constant=1.0,
                                                angle_softening_constant=1.0,
                                                neglect_angles = False,
                                                use_14_nonbondeds = True)


        # Run pipeline...
        htfs = []
        for is_complex, (top, pos, sys) in enumerate(inputs):
            if not is_complex:
                point_mutation_engine = PointMutationEngine(wildtype_topology=top,
                                                                     system_generator=self.system_generator,
                                                                     chain_id=mutation_chain_id, # Denote the chain id allowed to mutate (it's always a string variable)
                                                                     max_point_mutants=1,
                                                                     residues_allowed_to_mutate=[mutation_residue_id], # The residue ids allowed to mutate
                                                                     allowed_mutations=[(mutation_residue_id, proposed_residue)], # The residue ids allowed to mutate with the three-letter code allowed to change
                                                                     aggregate=True) # Always allow aggregation

                topology_proposal = point_mutation_engine.propose(sys, top)

                # Fix naked charges in old and new systems
                old_topology_atom_map = {atom.index: atom.residue.name for atom in topology_proposal.old_topology.atoms()}
                new_topology_atom_map = {atom.index: atom.residue.name for atom in topology_proposal.new_topology.atoms()}
                for i, system in enumerate([topology_proposal.old_system, topology_proposal.new_system]):
                    force_dict = {i.__class__.__name__: i for i in system.getForces()}
                    atom_map = old_topology_atom_map if i == 0 else new_topology_atom_map
                    if 'NonbondedForce' in [k for k in force_dict.keys()]:
                        nb_force = force_dict['NonbondedForce']
                        for idx in range(nb_force.getNumParticles()):
                            if atom_map[idx] in ['HOH', 'WAT']: # Do not add naked charge fix to water hydrogens
                                continue
                            charge, sigma, epsilon = nb_force.getParticleParameters(idx)
                            if sigma == 0*unit.nanometer:
                                sigma = 0.06*unit.nanometer
                                nb_force.setParticleParameters(idx, charge, sigma, epsilon)
                            if epsilon == 0*unit.kilojoule_per_mole:
                                epsilon = 0.0001*unit.kilojoule_per_mole
                                nb_force.setParticleParameters(idx, charge, sigma, epsilon)
                self.topology_proposal = topology_proposal
                self.pos = pos
    

In [21]:
solvent_delivery = PointMutationExecutor2("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/input/rbd_protonated.pdb",
                        '1',
                        "501",
                        "GLN",
                        forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml', '/home/zhangi/choderalab/openmmforcefields/amber/ffxml/GLYCAM_06j-1.xml'],
                        ionic_strength=0.15*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                        )

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:setup:solvating at 0.15 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG, atomic number=6), Atom(name=ND2, atomic number=7), Atom(name=OD1, atomic number=8), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1), Atom(name=HD21, atomic number=1), Atom(name=HD22, atomic number=1)]
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG, atomic number=6), Atom(name=CD, atomic number=6), Atom(name=NE2, atomic number=7), Atom(name=OE1, atomic number=8), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1), Atom(name=HG2, atomic number=1), Atom(name=HG3, atomic number=1), Atom(name=HE21, atomic number=1), Atom(name=HE22, atomic number=1)]


In [22]:
topology_proposal = solvent_delivery.topology_proposal

In [28]:
pos = solvent_delivery.pos

In [31]:
with open("N501Q_apo_topology_proposal.pickle", "wb") as f:
    pickle.dump(topology_proposal, f)

with open("N501Q_apo_pos.pickle", "wb") as f:
    pickle.dump(pos, f)

In [4]:
with open("N501Q_apo_topology_proposal.pickle", "rb") as f:
    topology_proposal = pickle.load(f)

with open("N501Q_apo_pos.pickle", "rb") as f:
    pos = pickle.load(f)

In [34]:
for atom in topology_proposal.old_topology.atoms():
    if atom.residue.id == '501':
        print(atom)

<Atom 2605 (N) of chain 0 residue 169 (ASN)>
<Atom 2606 (H) of chain 0 residue 169 (ASN)>
<Atom 2607 (CA) of chain 0 residue 169 (ASN)>
<Atom 2608 (HA) of chain 0 residue 169 (ASN)>
<Atom 2609 (C) of chain 0 residue 169 (ASN)>
<Atom 2610 (O) of chain 0 residue 169 (ASN)>
<Atom 2611 (CB) of chain 0 residue 169 (ASN)>
<Atom 2612 (HB2) of chain 0 residue 169 (ASN)>
<Atom 2613 (HB3) of chain 0 residue 169 (ASN)>
<Atom 2614 (CG) of chain 0 residue 169 (ASN)>
<Atom 2615 (ND2) of chain 0 residue 169 (ASN)>
<Atom 2616 (HD21) of chain 0 residue 169 (ASN)>
<Atom 2617 (HD22) of chain 0 residue 169 (ASN)>
<Atom 2618 (OD1) of chain 0 residue 169 (ASN)>
<Atom 4114 (O) of chain 2 residue 499 (HOH)>
<Atom 4115 (H1) of chain 2 residue 499 (HOH)>
<Atom 4116 (H2) of chain 2 residue 499 (HOH)>


In [35]:
for atom in topology_proposal.new_topology.atoms():
    if atom.residue.id == '501':
        print(atom)

<Atom 2605 (N) of chain 0 residue 169 (GLN)>
<Atom 2606 (H) of chain 0 residue 169 (GLN)>
<Atom 2607 (CA) of chain 0 residue 169 (GLN)>
<Atom 2608 (HA) of chain 0 residue 169 (GLN)>
<Atom 2609 (C) of chain 0 residue 169 (GLN)>
<Atom 2610 (O) of chain 0 residue 169 (GLN)>
<Atom 2611 (CB) of chain 0 residue 169 (GLN)>
<Atom 2612 (HB2) of chain 0 residue 169 (GLN)>
<Atom 2613 (HB3) of chain 0 residue 169 (GLN)>
<Atom 2614 (CG) of chain 0 residue 169 (GLN)>
<Atom 2615 (HG2) of chain 0 residue 169 (GLN)>
<Atom 2616 (HG3) of chain 0 residue 169 (GLN)>
<Atom 2617 (CD) of chain 0 residue 169 (GLN)>
<Atom 2618 (OE1) of chain 0 residue 169 (GLN)>
<Atom 2619 (NE2) of chain 0 residue 169 (GLN)>
<Atom 2620 (HE21) of chain 0 residue 169 (GLN)>
<Atom 2621 (HE22) of chain 0 residue 169 (GLN)>
<Atom 4117 (O) of chain 2 residue 499 (HOH)>
<Atom 4118 (H1) of chain 2 residue 499 (HOH)>
<Atom 4119 (H2) of chain 2 residue 499 (HOH)>


In [26]:
topology_proposal._core_new_to_old_atom_map

{2605: 2605,
 2607: 2607,
 2609: 2609,
 2610: 2610,
 2606: 2606,
 2608: 2608,
 2617: 2614,
 2619: 2615,
 2618: 2618,
 2621: 2616,
 2620: 2617,
 2591: 2591,
 2592: 2592,
 2593: 2593,
 2594: 2594,
 2595: 2595,
 2596: 2596,
 2597: 2597,
 2598: 2598,
 2599: 2599,
 2600: 2600,
 2601: 2601,
 2602: 2602,
 2603: 2603,
 2604: 2604,
 2622: 2619,
 2623: 2620,
 2624: 2621,
 2625: 2622,
 2626: 2623,
 2627: 2624,
 2628: 2625}

In [32]:
 geometry_engine = FFAllAngleGeometryEngine(metadata=None,
                                                use_sterics=False,
                                                n_bond_divisions=100,
                                                n_angle_divisions=180,
                                                n_torsion_divisions=360,
                                                verbose=True,
                                                storage=None,
                                                bond_softening_constant=1.0,
                                                angle_softening_constant=1.0,
                                                neglect_angles = False,
                                                use_14_nonbondeds = True)

In [33]:
# Only validate energy bookkeeping if the WT and proposed residues do not involve rings
validate_bool=True
new_positions, logp_proposal = geometry_engine.propose(topology_proposal, pos, beta,
                                                       validate_energy_bookkeeping=validate_bool)
# logp_reverse = geometry_engine.logp_reverse(topology_proposal, new_positions, pos, beta,
#                                             validate_energy_bookkeeping=validate_bool)



INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 6
INFO:geometry:Atom index proposal order is [2611, 2614, 2613, 2612, 2615, 2616]
INFO:geometry:omitted_bonds: [(2614, 2617)]
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 1720 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 5918 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 11214 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 6 new atoms
INFO:geometry:	reduced angle potential = 0.0027324040777624917.
INFO:geometry:	reduced angle potential = 0.0751540410589992.
INFO:geometry:	reduced angle potential = 0.24051736342879748.
INFO:geometry:	reduced angle potential = 0.14777590275060148.
INFO:geometry:	reduced angle potential = 0.586551768417793.
INFO:geometry:	reduced angle potential = 0.003853281997360432.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 1720 bond forces in the no-nonbonded final system
INFO:geometry:	there are 5918 angle forces in the no-nonbonded final system
INFO:geometry:	there are 11214 torsion forces in the no-

...

INFO:geometry:total reduced potential before atom placement: 4816.486937715517


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 596.4638800232323
INFO:geometry:final reduced energy 693702631.1902328
INFO:geometry:sum of energies: 5412.95081773875
INFO:geometry:magnitude of difference in the energies: 693697218.2394149


added energy components: [('CustomBondForce', 1.833636600999181), ('CustomAngleForce', 339.0093316617409), ('CustomTorsionForce', 27.665386834610043), ('CustomBondForce', 227.95552492588212)]


AssertionError: The ratio of the calculated final energy to the true final energy is 7.802984411997086e-06

In [8]:
solvent_delivery = PointMutationExecutor2("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/input/rbd_protonated.pdb",
                        '1',
                        "501",
                        "GLY",
                        forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml', '/home/zhangi/choderalab/openmmforcefields/amber/ffxml/GLYCAM_06j-1.xml'],
                        ionic_strength=0.15*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                        )

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:setup:solvating at 0.15 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG, atomic number=6), Atom(name=ND2, atomic number=7), Atom(name=OD1, atomic number=8), Atom(name=HA, atomic number=1), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1), Atom(name=HD21, atomic number=1), Atom(name=HD22, atomic number=1)]
INFO:proposal_generator:[Atom(name=HA3, atomic number=1), Atom(name=HA2, atomic number=1)]


In [9]:
topology_proposal = solvent_delivery.topology_proposal

In [10]:
pos = solvent_delivery.pos

In [11]:
with open("N501G_apo_topology_proposal.pickle", "wb") as f:
    pickle.dump(topology_proposal, f)

with open("N501G_apo_pos.pickle", "wb") as f:
    pickle.dump(pos, f)

In [34]:
for atom in topology_proposal.old_topology.atoms():
    if atom.residue.id == '501':
        print(atom)

<Atom 2605 (N) of chain 0 residue 169 (ASN)>
<Atom 2606 (H) of chain 0 residue 169 (ASN)>
<Atom 2607 (CA) of chain 0 residue 169 (ASN)>
<Atom 2608 (HA) of chain 0 residue 169 (ASN)>
<Atom 2609 (C) of chain 0 residue 169 (ASN)>
<Atom 2610 (O) of chain 0 residue 169 (ASN)>
<Atom 2611 (CB) of chain 0 residue 169 (ASN)>
<Atom 2612 (HB2) of chain 0 residue 169 (ASN)>
<Atom 2613 (HB3) of chain 0 residue 169 (ASN)>
<Atom 2614 (CG) of chain 0 residue 169 (ASN)>
<Atom 2615 (ND2) of chain 0 residue 169 (ASN)>
<Atom 2616 (HD21) of chain 0 residue 169 (ASN)>
<Atom 2617 (HD22) of chain 0 residue 169 (ASN)>
<Atom 2618 (OD1) of chain 0 residue 169 (ASN)>
<Atom 4114 (O) of chain 2 residue 499 (HOH)>
<Atom 4115 (H1) of chain 2 residue 499 (HOH)>
<Atom 4116 (H2) of chain 2 residue 499 (HOH)>


In [35]:
for atom in topology_proposal.new_topology.atoms():
    if atom.residue.id == '501':
        print(atom)

<Atom 2605 (N) of chain 0 residue 169 (GLN)>
<Atom 2606 (H) of chain 0 residue 169 (GLN)>
<Atom 2607 (CA) of chain 0 residue 169 (GLN)>
<Atom 2608 (HA) of chain 0 residue 169 (GLN)>
<Atom 2609 (C) of chain 0 residue 169 (GLN)>
<Atom 2610 (O) of chain 0 residue 169 (GLN)>
<Atom 2611 (CB) of chain 0 residue 169 (GLN)>
<Atom 2612 (HB2) of chain 0 residue 169 (GLN)>
<Atom 2613 (HB3) of chain 0 residue 169 (GLN)>
<Atom 2614 (CG) of chain 0 residue 169 (GLN)>
<Atom 2615 (HG2) of chain 0 residue 169 (GLN)>
<Atom 2616 (HG3) of chain 0 residue 169 (GLN)>
<Atom 2617 (CD) of chain 0 residue 169 (GLN)>
<Atom 2618 (OE1) of chain 0 residue 169 (GLN)>
<Atom 2619 (NE2) of chain 0 residue 169 (GLN)>
<Atom 2620 (HE21) of chain 0 residue 169 (GLN)>
<Atom 2621 (HE22) of chain 0 residue 169 (GLN)>
<Atom 4117 (O) of chain 2 residue 499 (HOH)>
<Atom 4118 (H1) of chain 2 residue 499 (HOH)>
<Atom 4119 (H2) of chain 2 residue 499 (HOH)>


In [26]:
topology_proposal._core_new_to_old_atom_map

{2605: 2605,
 2607: 2607,
 2609: 2609,
 2610: 2610,
 2606: 2606,
 2608: 2608,
 2617: 2614,
 2619: 2615,
 2618: 2618,
 2621: 2616,
 2620: 2617,
 2591: 2591,
 2592: 2592,
 2593: 2593,
 2594: 2594,
 2595: 2595,
 2596: 2596,
 2597: 2597,
 2598: 2598,
 2599: 2599,
 2600: 2600,
 2601: 2601,
 2602: 2602,
 2603: 2603,
 2604: 2604,
 2622: 2619,
 2623: 2620,
 2624: 2621,
 2625: 2622,
 2626: 2623,
 2627: 2624,
 2628: 2625}

In [12]:
solvent_delivery = PointMutationExecutor2("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/input/rbd_protonated.pdb",
                        '1',
                        "501",
                        "ARG",
                        forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml', '/home/zhangi/choderalab/openmmforcefields/amber/ffxml/GLYCAM_06j-1.xml'],
                        ionic_strength=0.15*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                        )

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:setup:solvating at 0.15 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG, atomic number=6), Atom(name=ND2, atomic number=7), Atom(name=OD1, atomic number=8), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1), Atom(name=HD21, atomic number=1), Atom(name=HD22, atomic number=1)]
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG, atomic number=6), Atom(name=CD, atomic number=6), Atom(name=NE, atomic number=7), Atom(name=CZ, atomic number=6), Atom(name=NH1, atomic number=7), Atom(name=NH2, atomic number=7), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1), Atom(name=HG2, atomic number=1), Atom(name=HG3, atomic number=1), Atom(name=HD2, atomic number=1), Atom(name=HD3, atomic number=1), Atom(name=HE, atomic number=1), Atom(name=HH11, atomic numb

In [13]:
topology_proposal = solvent_delivery.topology_proposal

In [14]:
pos = solvent_delivery.pos

In [15]:
with open("N501R_apo_topology_proposal.pickle", "wb") as f:
    pickle.dump(topology_proposal, f)

with open("N501R_apo_pos.pickle", "wb") as f:
    pickle.dump(pos, f)

In [16]:
for atom in topology_proposal.old_topology.atoms():
    if atom.residue.id == '501':
        print(atom)

<Atom 2605 (N) of chain 0 residue 169 (ASN)>
<Atom 2606 (H) of chain 0 residue 169 (ASN)>
<Atom 2607 (CA) of chain 0 residue 169 (ASN)>
<Atom 2608 (HA) of chain 0 residue 169 (ASN)>
<Atom 2609 (C) of chain 0 residue 169 (ASN)>
<Atom 2610 (O) of chain 0 residue 169 (ASN)>
<Atom 2611 (CB) of chain 0 residue 169 (ASN)>
<Atom 2612 (HB2) of chain 0 residue 169 (ASN)>
<Atom 2613 (HB3) of chain 0 residue 169 (ASN)>
<Atom 2614 (CG) of chain 0 residue 169 (ASN)>
<Atom 2615 (ND2) of chain 0 residue 169 (ASN)>
<Atom 2616 (HD21) of chain 0 residue 169 (ASN)>
<Atom 2617 (HD22) of chain 0 residue 169 (ASN)>
<Atom 2618 (OD1) of chain 0 residue 169 (ASN)>
<Atom 4108 (O) of chain 2 residue 497 (HOH)>
<Atom 4109 (H1) of chain 2 residue 497 (HOH)>
<Atom 4110 (H2) of chain 2 residue 497 (HOH)>


In [17]:
for atom in topology_proposal.new_topology.atoms():
    if atom.residue.id == '501':
        print(atom)

<Atom 2605 (N) of chain 0 residue 169 (ARG)>
<Atom 2606 (H) of chain 0 residue 169 (ARG)>
<Atom 2607 (CA) of chain 0 residue 169 (ARG)>
<Atom 2608 (HA) of chain 0 residue 169 (ARG)>
<Atom 2609 (C) of chain 0 residue 169 (ARG)>
<Atom 2610 (O) of chain 0 residue 169 (ARG)>
<Atom 2611 (CB) of chain 0 residue 169 (ARG)>
<Atom 2612 (HB2) of chain 0 residue 169 (ARG)>
<Atom 2613 (HB3) of chain 0 residue 169 (ARG)>
<Atom 2614 (CG) of chain 0 residue 169 (ARG)>
<Atom 2615 (HG2) of chain 0 residue 169 (ARG)>
<Atom 2616 (HG3) of chain 0 residue 169 (ARG)>
<Atom 2617 (CD) of chain 0 residue 169 (ARG)>
<Atom 2618 (HD2) of chain 0 residue 169 (ARG)>
<Atom 2619 (HD3) of chain 0 residue 169 (ARG)>
<Atom 2620 (NE) of chain 0 residue 169 (ARG)>
<Atom 2621 (HE) of chain 0 residue 169 (ARG)>
<Atom 2622 (CZ) of chain 0 residue 169 (ARG)>
<Atom 2623 (NH1) of chain 0 residue 169 (ARG)>
<Atom 2624 (HH11) of chain 0 residue 169 (ARG)>
<Atom 2625 (HH12) of chain 0 residue 169 (ARG)>
<Atom 2626 (NH2) of chain 0

In [18]:
topology_proposal._core_new_to_old_atom_map

{2605: 2605,
 2607: 2607,
 2609: 2609,
 2610: 2610,
 2606: 2606,
 2608: 2608,
 2622: 2614,
 2623: 2615,
 2625: 2616,
 2624: 2617,
 2591: 2591,
 2592: 2592,
 2593: 2593,
 2594: 2594,
 2595: 2595,
 2596: 2596,
 2597: 2597,
 2598: 2598,
 2599: 2599,
 2600: 2600,
 2601: 2601,
 2602: 2602,
 2603: 2603,
 2604: 2604,
 2629: 2619,
 2630: 2620,
 2631: 2621,
 2632: 2622,
 2633: 2623,
 2634: 2624,
 2635: 2625}

# Figure out why atom map for N501Q does not contain CB or HBs but does contain other sidechain atoms

In [2]:
class AtomMapper2(AtomMapper):
    def get_best_mapping(self, old_mol, new_mol):
        """Retrieve the best mapping between old and new molecules.
        .. note ::
           This method may generate multiple distinct mappings with the same best score;
           the choice of mapping is returned is ambiguous.
         .. todo ::
           We should figure out how to make the choice deterministic in the case
           multiple mappings have the same score.
        Parameters
        ----------
        old_mol : openff.toolkit.topology.Molecule or openeye.oechem.OEMol
            The initial molecule for the transformation.
        new_mol : openff.toolkit.topology.Molecule or openeye.oechem.OEMol
            The final molecule for the transformation.
        Returns
        -------
        atom_mapping : AtomMapping
            Atom mapping with the best score
        Examples
        --------
        Retrieve best-scoring mapping between ethane and ethanol
        >>> from openff.toolkit.topology import Molecule
        >>> ethane = Molecule.from_smiles('[C:0]([H:1])([H:2])([H:3])[C:4]([H:5])([H:6])([H:7])')
        >>> ethanol = Molecule.from_smiles('[C:0]([H:1])([H:2])([H:3])[C:4]([H:5])([H:6])[O:7][H:8]')
        >>> atom_mapping = atom_mapper.get_best_mapping(ethane, ethanol)
        """
        import time
        initial_time = time.time()

        import numpy as np
        atom_mappings = self.get_all_mappings(old_mol, new_mol)
        if (atom_mappings is None) or len(atom_mappings)==0:
            return None

        scores = np.array([ self.score_mapping(atom_mapping) for atom_mapping in atom_mappings ])
        best_map_index = np.argmax(scores)

        elapsed_time = time.time() - initial_time
        _logger.debug(f'get_best_mapping took {elapsed_time:.3f} s')

        return atom_mappings[best_map_index]

NameError: name 'AtomMapper' is not defined

In [ ]:
from perses.rjmc.topology_proposal import append_topology, PolymerProposalEngine, PointMutationEngine, SmallMoleculeSetProposalEngine, augment_openmm_topology, TopologyProposal

from simtk.openmm import app

import copy
import logging
import itertools
import os
import numpy as np
import networkx as nx
import openmoltools.forcefield_generators as forcefield_generators
from perses.storage import NetCDFStorageView
from perses.rjmc.geometry import NoTorsionError
from functools import partial
from simtk import unit # needed for unit-bearing quantity defaults
try:
    from subprocess import getoutput  # If python 3
except ImportError:
    from commands import getoutput  # If python 2

################################################################################
# LOGGER
################################################################################

import logging
logging.basicConfig(level = logging.NOTSET)
_logger = logging.getLogger("proposal_generator")
_logger.setLevel(logging.INFO)

class PointMutationEngine2(PointMutationEngine):
    def propose(self,
                current_system,
                current_topology,
                current_metadata=None):
        """
        Generate a TopologyProposal
        Parameters
        ----------
        current_system : simtk.openmm.System object
            The current system object
        current_topology : simtk.openmm.app.Topology object
            The current topology
        current_metadata : dict -- OPTIONAL
        Returns
        -------
        topology_proposal : TopologyProposal
            NamedTuple of type TopologyProposal containing forward and reverse
            probabilities, as well as old and new topologies and atom
            mapping
        """
        _logger.info(f"\tConducting polymer point mutation proposal...")
        old_topology = app.Topology()
        append_topology(old_topology, current_topology)

        # new_topology : simtk.openmm.app.Topology
        new_topology = app.Topology()
        append_topology(new_topology, current_topology)

        # Check that old_topology and old_system have same number of atoms.
        old_system = current_system
        old_topology_natoms = old_topology.getNumAtoms()  # number of topology atoms
        old_system_natoms = old_system.getNumParticles()
        if old_topology_natoms != old_system_natoms:
            msg = 'PolymerProposalEngine: old_topology has %d atoms, while old_system has %d atoms' % (old_topology_natoms, old_system_natoms)
            raise Exception(msg)

        # metadata : dict, key = 'chain_id' , value : str
        metadata = current_metadata
        if metadata is None:
            metadata = dict()

        # old_chemical_state_key : str
        _logger.debug(f"\tcomputing state key of old topology...")
        old_chemical_state_key = self.compute_state_key(old_topology)
        _logger.debug(f"\told chemical state key for chain {self._chain_id}: {old_chemical_state_key}")

        # index_to_new_residues : dict, key : int (index) , value : str (three letter name of proposed residue)
        _logger.debug(f"\tchoosing mutant...")
        index_to_new_residues, metadata = self._choose_mutant(old_topology, metadata)
        _logger.debug(f"\t\tindex to new residues: {index_to_new_residues}")

        # residue_map : list(tuples : simtk.openmm.app.topology.Residue (existing residue), str (three letter name of proposed residue))
        _logger.debug(f"\tgenerating residue map...")
        residue_map = self._generate_residue_map(old_topology, index_to_new_residues)
        _logger.debug(f"\t\tresidue map: {residue_map}")

        for (res, new_name) in residue_map:
            if res.name == new_name:
                #remove the index_to_new_residues entries where the topology is already mutated
                del(index_to_new_residues[res.index])
        if len(index_to_new_residues) == 0:
            _logger.debug(f"\t\tno mutation detected in this proposal; generating old proposal")
            atom_map = dict()
            for atom in old_topology.atoms():
                atom_map[atom.index] = atom.index
            _logger.debug('PolymerProposalEngine: No changes to topology proposed, returning old system and topology')
            topology_proposal = TopologyProposal(new_topology=old_topology, new_system=old_system, old_topology=old_topology, old_system=old_system, old_chemical_state_key=old_chemical_state_key, new_chemical_state_key=old_chemical_state_key, logp_proposal=0.0, new_to_old_atom_map=atom_map)
            return topology_proposal

        elif len(index_to_new_residues) > 1:
            raise Exception("Attempting to mutate more than one residue at once: ", index_to_new_residues, " The geometry engine cannot handle this.")

        chosen_res_index = list(index_to_new_residues.keys())[0]
        # Add modified_aa property to residues in old topology
        for res in old_topology.residues():
            res.modified_aa = True if res.index in index_to_new_residues.keys() else False

        _logger.debug(f"\tfinal index_to_new_residues: {index_to_new_residues}")
        _logger.debug(f"\tfinding excess and missing atoms/bonds...")
        # Identify differences between old topology and proposed changes
        # excess_atoms : list(simtk.openmm.app.topology.Atom) atoms from existing residue not in new residue
        # excess_bonds : list(tuple (simtk.openmm.app.topology.Atom, simtk.openmm.app.topology.Atom)) bonds from existing residue not in new residue
        # missing_bonds : list(tuple (simtk.openmm.app.topology._TemplateAtomData, simtk.openmm.app.topology._TemplateAtomData)) bonds from new residue not in existing residue
        excess_atoms, excess_bonds, missing_atoms, missing_bonds = self._identify_differences(old_topology, residue_map)

        # Delete excess atoms and bonds from old topology
        excess_atoms_bonds = excess_atoms + excess_bonds
        _logger.debug(f"\t excess atoms bonds: {excess_atoms_bonds}")
        new_topology = self._delete_atoms(old_topology, excess_atoms_bonds)

        # Add missing atoms and bonds to new topology
        new_topology = self._add_new_atoms(new_topology, missing_atoms, missing_bonds, residue_map)

        # index_to_new_residues : dict, key : int (index) , value : str (three letter name of proposed residue)
        _logger.debug(f"\tconstructing atom map for TopologyProposal...")
        atom_map, old_res_to_oemol_map, new_res_to_oemol_map, old_oemol_res_copy, new_oemol_res_copy  = self._construct_atom_map(residue_map, old_topology, new_topology)
#         atom_map, old_res_to_oemol_map, new_res_to_oemol_map, local_atom_map_stereo_sidechain, current_oemol_sidechain, proposed_oemol_sidechain, old_oemol_res_copy, new_oemol_res_copy  = self._construct_atom_map(residue_map, old_topology, index_to_new_residues, new_topology, ignore_sidechain_atoms=False)
        print("HERE's the atom map from construct_atom_map()", atom_map)
        
        _logger.debug(f"\tadding indices of the 'C' backbone atom in the next residue and the 'N' atom in the previous")
        _logger.debug(f"\t{list(index_to_new_residues.keys())[0]}")
        extra_atom_map = self._find_adjacent_residue_atoms(old_topology, new_topology, list(index_to_new_residues.keys())[0])
        _logger.debug(f"\tfound extra atom map: {extra_atom_map}")

        #now to add all of the other residue atoms to the atom map...
        all_other_residues_new = [res for res in new_topology.residues() if res.index != list(index_to_new_residues.keys())[0]]
        all_other_residues_old = [res for res in old_topology.residues() if res.index != list(index_to_new_residues.keys())[0]]

        all_other_atoms_map = {}
        for res_new, res_old in zip(all_other_residues_new, all_other_residues_old):
            assert res_new.name == res_old.name, f"all other residue names do not match"
            all_other_atoms_map.update({atom_new.index: atom_old.index for atom_new, atom_old in zip(res_new.atoms(), res_old.atoms())})
            
        # new_chemical_state_key : str
        new_chemical_state_key = self.compute_state_key(new_topology)
        # new_system : simtk.openmm.System

        # Copy periodic box vectors from current topology
        new_topology.setPeriodicBoxVectors(current_topology.getPeriodicBoxVectors())

        # Build system
        # TODO: Remove build_system() branch once we convert entirely to new openmm-forcefields SystemBuilder
        if hasattr(self._system_generator, 'create_system'):
            new_system = self._system_generator.create_system(new_topology)
        else:
            new_system = self._system_generator.build_system(new_topology)

        # Explicitly de-map any atoms involved in constraints that change length
        atom_map = SmallMoleculeSetProposalEngine._constraint_repairs(atom_map, old_system, new_system, old_topology, new_topology)
        _logger.debug(f"\tafter constraint repairs, the atom map is as such: {atom_map}")

        _logger.debug(f"\tadding all env atoms to the atom map...")
        atom_map.update(all_other_atoms_map)

        old_res_names = [res.name for res in old_topology.residues() if res.index == list(index_to_new_residues.keys())[0]]
        assert len(old_res_names) == 1, f"no old res name match found"
        old_res_name = old_res_names[0]
        _logger.debug(f"\told res name: {old_res_name}")
        new_res_name = list(index_to_new_residues.values())[0]

        # Create TopologyProposal.
        current_res = [res for res in current_topology.residues() if res.index == chosen_res_index][0]
        proposed_res = [res for res in new_topology.residues() if res.index == chosen_res_index][0]
        augment_openmm_topology(topology = old_topology, residue_oemol = old_oemol_res_copy, residue_topology = current_res, residue_to_oemol_map = old_res_to_oemol_map)
        augment_openmm_topology(topology = new_topology, residue_oemol = new_oemol_res_copy, residue_topology = proposed_res, residue_to_oemol_map = new_res_to_oemol_map)

        topology_proposal = TopologyProposal(logp_proposal = 0.,
                                             new_to_old_atom_map = atom_map,
                                             old_topology = old_topology,
                                             new_topology  = new_topology,
                                             old_system = old_system,
                                             new_system = new_system,
                                             old_alchemical_atoms = [atom.index for atom in current_res.atoms()] + list(extra_atom_map.values()),
                                             old_chemical_state_key = old_chemical_state_key,
                                             new_chemical_state_key = new_chemical_state_key,
                                             old_residue_name = old_res_name,
                                             new_residue_name = new_res_name)

        # Check that old_topology and old_system have same number of atoms.
        old_topology_natoms = old_topology.getNumAtoms()  # number of topology atoms
        old_system_natoms = old_system.getNumParticles()
        if old_topology_natoms != old_system_natoms:
            msg = 'PolymerProposalEngine: old_topology has %d atoms, while old_system has %d atoms' % (old_topology_natoms, old_system_natoms)
            raise Exception(msg)

        # Check that new_topology and new_system have same number of atoms.
        new_topology_natoms = new_topology.getNumAtoms()  # number of topology atoms
        new_system_natoms = new_system.getNumParticles()
        if new_topology_natoms != new_system_natoms:
            msg = 'PolymerProposalEngine: new_topology has %d atoms, while new_system has %d atoms' % (new_topology_natoms, new_system_natoms)
            raise Exception(msg)

        # Check to make sure no out-of-bounds atoms are present in new_to_old_atom_map
        natoms_old = topology_proposal.old_system.getNumParticles()
        natoms_new = topology_proposal.new_system.getNumParticles()
        if not set(topology_proposal.new_to_old_atom_map.values()).issubset(range(natoms_old)):
            msg = "Some new atoms in TopologyProposal.new_to_old_atom_map are not in span of new atoms (1..%d):\n" % natoms_new
            msg += str(topology_proposal.new_to_old_atom_map)
            raise Exception(msg)
        if not set(topology_proposal.new_to_old_atom_map.keys()).issubset(range(natoms_new)):
            msg = "Some new atoms in TopologyProposal.new_to_old_atom_map are not in span of old atoms (1..%d):\n" % natoms_new
            msg += str(topology_proposal.new_to_old_atom_map)
            raise Exception(msg)

        #validate the old/new system matches
        # TODO: create more rigorous checks for this validation either in TopologyProposal or in the HybridTopologyFactory
        #assert PolymerProposalEngine.validate_core_atoms_with_system(topology_proposal)


        return topology_proposal
   
    def _construct_atom_map(self,
                            residue_map,
                            old_topology,
                            new_topology,
                           extra_sidechain_map=None):
        """
        Construct atom map (key: index to atom in new residue, value: index to atom in old residue) to supply as an argument to the TopologyProposal.
        
        By default, the atom map:
        1) Maps all backbone atoms (exception: for GLY, do not map HA2 and HA3)
        2) Map CBs and HBs only (no other sidechain atoms)
        
        If additional sidechains should be mapped, extra_sidechain_map can be supplied to supplement the default map.
        
        Parameters
        ----------
        residue_map : list(tuples)
            simtk.openmm.app.topology.Residue, str (three letter residue name of new residue)
        old_topology : simtk.openmm.app.Topology
            topology of old system
        new_topology : simtk.openmm.app.Topology
            topology of new system
        extra_sidechain_map : dict, key: int, value: int, default None
            map of new to old sidechain atom indices to add to the local_atom_map

        Returns
        -------
        local_atom_map : dict, key: int, value int
            mapping of atom indices in new residue to indices in old residue
        old_res_to_oemol_map : dict, key: int, value int
            mapping of OpenMM topology to oemol indices for old residue
        new_res_to_oemol_map : dict, key: int, value int
            mapping of OpenMM topology to oemol indices for new residue
        old_oemol : openeye.oechem.oemol object
            old oemol
        new_oemol : openeye.oechem.oemol object
            new oemol
        .. todo ::
            * Move this into atom_mapping.py
            * Overhaul biopolymer mapping to use openff.topology.Topology features
            * Generalize to support arbitrary biopolymer residues and protonation/tautomeric states
        """
        from pkg_resources import resource_filename
        import openeye.oechem as oechem
        
        # Retrieve map of old to new residues
        # old_to_new_residues : dict, key : simtk.openmm.app.topology.Residue old residue, value : simtk.openmm.app.topology.Residue new residue
        old_to_new_residues = {}
        new_residues = [residue for residue in new_topology.residues()] # Assumes all residue indices start from 0 and are contiguous
        for old_residue in old_topology.residues():
            old_to_new_residues[old_residue] = new_residues[old_residue.index]

        # Retrieve old and new residues
        assert len(residue_map) == 1, f"residue_map is not of length 1"
        old_res = residue_map[0][0]
        new_res = old_to_new_residues[old_res]
        old_res_name = old_res.name
        new_res_name = new_res.name

        # Retrieve map of atom index to atom name for old and new residues
        # old_res_index_to_name : dict, key : int old atom index, value : str old atom name
        # new_res_index_to_name : dict, key : int new atom index, value : str new atom name
        old_res_index_to_name = {atom.index: atom.name for atom in old_res.atoms()}
        new_res_index_to_name = {atom.index: atom.name for atom in new_res.atoms()}

        # Retrieve map of atom name to atom index for old and new residues
        # old_res_name_to_index : dict, key : str old atom name, value : int old atom index
        # new_res_name_to_index : dict, key : str new atom name, value : int new atom index
        old_res_name_to_index = {atom.name: atom.index for atom in old_res.atoms()}
        new_res_name_to_index = {atom.name: atom.index for atom in new_res.atoms()}

        # Initialize_the atom map
        local_atom_map = {}

        # Iterate over the old res atoms to fill in the local atom map
        backbone_atoms = ['C', 'CA', 'N', 'O', 'H', 'HA']
        sidechain_atoms = ['CB', 'HB', 'HB2', 'HB3']
        for atom in old_res.atoms():
            old_atom_index = atom.index
            old_atom_name = atom.name
            
            if old_atom_name in backbone_atoms:
                if old_res_name == 'GLY' and old_atom_name in ['HA2', 'HA3']: # Do not map HA if old residue GLY
                    continue
                elif new_res_name == 'GLY' and old_atom_name == 'HA': # Do not map HA if new residue is GLY
                    continue
                else:
                    new_atom_index = new_res_name_to_index[old_atom_name]
                    local_atom_map[new_atom_index] = old_atom_index
            
            elif old_atom_name in sidechain_atoms:
                if new_res_name == 'GLY': # Do not map sidechain atoms if GLY
                    continue
                elif old_res_name in ['THR', 'VAL', 'ILE'] and new_res_name not in ['THR', 'VAL', 'ILE']: # If old residue contains HB and new residue contains HB2/HB3
                    if old_atom_name == 'HB':
                        new_atom_index = new_res_name_to_index[old_atom_name + '2'] # Map HB to HB2
                        local_atom_map[new_atom_index] = old_atom_index
                    elif old_atom_name == 'CB':
                        new_atom_index = new_res_name_to_index[old_atom_name]
                        local_atom_map[new_atom_index] = old_atom_index
                elif old_res_name not in ['THR', 'VAL', 'ILE'] and new_res_name in ['THR', 'VAL', 'ILE']: # If old residue contains HB2/HB3 and new residue contains HB
                    if old_atom_name == 'HB2':
                        new_atom_index = new_res_name_to_index[old_atom_name[:-1]] # Map HB2 to HB
                        local_atom_map[new_atom_index] = old_atom_index
                    elif old_atom_name == 'CB':
                        new_atom_index = new_res_name_to_index[old_atom_name]
                        local_atom_map[new_atom_index] = old_atom_index
                else:
                    new_atom_index = new_res_name_to_index[old_atom_name]
                    local_atom_map[new_atom_index] = old_atom_index
        
        # Validate extra_sidechain_map
        if extra_sidechain_map:
            assert type(extra_sidechain_map) is dict, "extra_sidechain_map must be a dict"
            old_res_indices = [atom.index for atom in old_res.atoms()]
            new_res_indices = [atom.index for atom in new_res.atoms()]
            assert all([index in new_res_indices for index in extra_sidechain_map.keys()]), "at least one of the new indices in extra_sidechain_map is not present in the new_topology"
            assert all([index in old_res_indices for index in extra_sidechain_map.values()]), "at least one of the new indices in extra_sidechain_map is not present in the old_topology"
        
            # Add extra_sidechain_map to local_atom_map
            local_atom_map.update(extra_sidechain_map)
        
        _logger.info(f"local_atom_map: {local_atom_map}")
        
        mapped_atoms = [(new_res_index_to_name[new_idx], old_res_index_to_name[old_idx]) for new_idx, old_idx in local_atom_map.items()]
        _logger.info(f"the mapped atom names are: {mapped_atoms}")            
            
        # Retrieve old and new oemols
        old_residue_pdb_filename = resource_filename('perses', os.path.join('data', 'amino_acid_templates', f"{old_res_name}.pdb"))
        new_residue_pdb_filename = resource_filename('perses', os.path.join('data', 'amino_acid_templates', f"{new_res_name}.pdb"))
        old_oemol = PolymerProposalEngine.generate_oemol_from_pdb_template(old_residue_pdb_filename)
        new_oemol = PolymerProposalEngine.generate_oemol_from_pdb_template(new_residue_pdb_filename)

        # Retrieve mapping of OpenMM topology to oemol indices for old and new residues
        # old_res_to_oemol_map : dict, key : int old atom index (OpenMM topology), value : int old atom index (oemol)
        # new_res_to_oemol_map : dict, key : int new atom index (OpenMM topology), value : int new atom index (oemol)
        old_res_to_oemol_map = {atom.index: old_oemol.GetAtom(oechem.OEHasAtomName(atom.name)).GetIdx() for atom in old_res.atoms()}
        new_res_to_oemol_map = {atom.index: new_oemol.GetAtom(oechem.OEHasAtomName(atom.name)).GetIdx() for atom in new_res.atoms()}
            
        return local_atom_map, old_res_to_oemol_map, new_res_to_oemol_map, old_oemol, new_oemol
    
#     def _construct_atom_map(self,
#                             residue_map,
#                             old_topology,
#                             index_to_new_residues,
#                             new_topology,
#                             ignore_sidechain_atoms=False):
#         """
#         Construct atom map (key: index to new residue, value: index to old residue) to supply as an argument to the TopologyProposal.
#         Parameters
#         ----------
#         residue_map : list(tuples)
#             simtk.openmm.app.topology.Residue, str (three letter residue name of new residue)
#         old_topology : simtk.openmm.app.Topology
#             topology of old system
#         index_to_new_residues : dict
#             key : int (index) , value : str (three letter name of proposed residue)
#         new_topology : simtk.openmm.app.Topology
#             topology of new system
#         ignore_sidechain_atoms : bool, default False
#             whether to ignore sidechain atoms (CB atoms and beyond) in the atom map
#         Returns
#         -------
#         adjusted_atom_map : dict, key: int (index
#             new residues have all correct atoms and bonds for desired mutation
#         old_res_to_oemol_map : dict
#             key: int (index);  value: int (index)
#         new_res_to_oemol_map : dict
#             key: int (index);  value: int (index)
#         local_atom_map_stereo_sidechain : dict
#             chirality-corrected map of new_oemol_res to old_oemol_res
#         current_oemol : openeye.oechem.oemol object
#             copy of modified old oemol sidechain
#         proposed_oemol : openeye.oechem.oemol object
#             copy of modified new oemol sidechain
#         old_oemol_res_copy : openeye.oechem.oemol object
#             copy of modified old oemol
#         new_oemol_res_copy : openeye.oechem.oemol object
#             copy of modified new oemol
#         .. todo ::
#             * Move this into atom_mapping.py
#             * Overhaul biopolymer mapping to use openff.topology.Topology features
#             * Generalize to support arbitrary biopolymer residues and protonation/tautomeric states
#         """
#         from pkg_resources import resource_filename
#         import openeye.oechem as oechem #must this be explicit?

#         # atom_map : dict, key : int (index of atom in old topology) , value : int (index of same atom in new topology)
#         atom_map = dict()

#         # atoms with an old_index attribute should be mapped
#         # k : int
#         # atom : simtk.openmm.app.topology.Atom

#         # old_to_new_residues : dict, key : str old residue name, key : simtk.openmm.app.topology.Residue new residue
#         old_to_new_residues = {}
#         new_residues = [residue for residue in new_topology.residues()] # Assumes all residue indices start from 0 and are contiguous
#         for old_residue in old_topology.residues():
#             old_to_new_residues[old_residue] = new_residues[old_residue.index]
#         #_logger.debug(f"\t\told_to_new_residues: {old_to_new_residues}")

#         # modified_residues : dict, key : index of old residue, value : proposed residue
#         modified_residues = dict()

#         for map_entry in residue_map:
#             old_residue = map_entry[0]
#             modified_residues[old_residue.index] = old_to_new_residues[old_residue]
#         _logger.debug(f"\t\tmodified residues: {modified_residues}")

#         # old_residues : dict, key : index of old residue, value : old residue
#         old_residues = dict()
#         for residue in old_topology.residues():
#             if residue.index in index_to_new_residues.keys():
#                 old_residues[residue.index] = residue
#         _logger.debug(f"\t\t\told residues: {old_residues}")

#         # Update atom map with atom mappings for residues that have been modified
#         assert len(index_to_new_residues) == 1, f"index_to_new_residues is not of length 1"
#         index = list(index_to_new_residues.keys())[0]
#         #old_res = old_residues[index]
#         old_res = old_residues[index]
#         new_res = modified_residues[index]
#         _logger.debug(f"\t\t\told res: {old_res.name}; new res: {new_res.name}")

#         new_res_index_to_name = {atom.index: atom.name for atom in new_res.atoms()}
#         old_res_index_to_name = {atom.index: atom.name for atom in old_res.atoms()}

#         _logger.debug(f"\t\t\told topology res names: {old_res_index_to_name}")
#         _logger.debug(f"\t\t\tnew topology res names: {new_res_index_to_name}")

#         old_res_name = old_res.name
#         new_res_name = new_res.name

#         current_residue_pdb_filename = resource_filename('perses', os.path.join('data', 'amino_acid_templates', f"{old_res_name}.pdb"))
#         proposed_residue_pdb_filename = resource_filename('perses', os.path.join('data', 'amino_acid_templates', f"{new_res_name}.pdb"))

#         current_oemol = PolymerProposalEngine.generate_oemol_from_pdb_template(current_residue_pdb_filename)
#         proposed_oemol = PolymerProposalEngine.generate_oemol_from_pdb_template(proposed_residue_pdb_filename)

#         old_oemol_res_copy = copy.deepcopy(current_oemol)
#         new_oemol_res_copy = copy.deepcopy(proposed_oemol)


#         _logger.debug(f"\t\t\told_oemol_res names: {[(atom.GetIdx(), atom.GetName()) for atom in current_oemol.GetAtoms()]}")
#         _logger.debug(f"\t\t\tnew_oemol_res names: {[(atom.GetIdx(), atom.GetName()) for atom in proposed_oemol.GetAtoms()]}")

#         #create bookkeeping dictionaries
#         old_res_to_oemol_map = {atom.index: current_oemol.GetAtom(oechem.OEHasAtomName(atom.name)).GetIdx() for atom in old_res.atoms()}
#         new_res_to_oemol_map = {atom.index: proposed_oemol.GetAtom(oechem.OEHasAtomName(atom.name)).GetIdx() for atom in new_res.atoms()}

#         old_oemol_name_idx = {atom.GetName(): atom.GetIdx() for atom in current_oemol.GetAtoms()}
#         new_oemol_name_idx = {atom.GetName(): atom.GetIdx() for atom in proposed_oemol.GetAtoms()}

#         _logger.debug(f"\t\t\told_res_to_oemol_map: {old_res_to_oemol_map}")
#         _logger.debug(f"\t\t\tnew_res_to_oemol_map: {new_res_to_oemol_map}")

#         old_oemol_to_res_map = {val: key for key, val in old_res_to_oemol_map.items()}
#         new_oemol_to_res_map = {val: key for key, val in new_res_to_oemol_map.items()}

#         # HBM - these don't seem to be used anywhere
#         #old_res_to_oemol_molecule_map = {atom.index: current_oemol.GetAtom(oechem.OEHasAtomName(atom.name)) for atom in old_res.atoms()}
#         #new_res_to_oemol_molecule_map = {atom.index: proposed_oemol.GetAtom(oechem.OEHasAtomName(atom.name)) for atom in new_res.atoms()}

#         #initialize_the atom map
#         local_atom_map = {}

#         #now remove backbones in both molecules and map them separately
#         # TODO: Why don't we just include backbone atoms and just enforce their mapping via external indices?
#         backbone_atoms = ['C', 'CA', 'N', 'O', 'H', 'HA', "H'"]
#         # TODO dom make this a seperate function
#         old_atoms_to_delete, new_atoms_to_delete = [], []
#         for atom in proposed_oemol.GetAtoms():
#             if atom.GetName() in backbone_atoms:
#                 try: #to get the backbone atom with the same naem in the old_oemol_res
#                     old_corresponding_backbones = [_atom for _atom in current_oemol.GetAtoms() if _atom.GetName() == atom.GetName()]
#                     if old_corresponding_backbones == []:
#                         #this is an exception when the old oemol res is a glycine.  if this is the case, then we do not map HA2 or HA3
#                         assert set(['HA2', 'HA3']).issubset([_atom.GetName() for _atom in current_oemol.GetAtoms()]), f"old oemol residue is not a GLY template"
#                         #we have to map HA3 to HA (old, new)
#                         old_corresponding_backbones = [_atom for _atom in current_oemol.GetAtoms() if _atom.GetName() == 'HA3' and atom.GetName() == 'HA']
#                     assert len(old_corresponding_backbones) == 1, f"there can only be one corresponding backbone in the old molecule; corresponding backbones: {[atom.GetName() for atom in old_corresponding_backbones]}"
#                     old_corresponding_backbone = old_corresponding_backbones[0]
#                     if not atom.GetName() == "H'": #throw out the extra H
#                         local_atom_map[atom.GetIdx()] = old_corresponding_backbone.GetIdx()
#                     old_atoms_to_delete.append(old_corresponding_backbone)
#                     new_atoms_to_delete.append(atom)
#                     assert proposed_oemol.DeleteAtom(atom), f"failed to delete new_oemol atom {atom}"
#                     assert current_oemol.DeleteAtom(old_corresponding_backbone), f"failed to delete old_oemol atom {old_corresponding_backbone}"
#                 except Exception as e:
#                     raise Exception(f"failed to map the backbone separately: {e}")

#         _logger.debug(f"\t\t\told_oemol_res names: {[(atom.GetIdx(), atom.GetName()) for atom in current_oemol.GetAtoms()]}")
#         _logger.debug(f"\t\t\tnew_oemol_res names: {[(atom.GetIdx(), atom.GetName()) for atom in proposed_oemol.GetAtoms()]}")

#         old_sidechain_oemol_indices_to_name = {atom.GetIdx(): atom.GetName() for atom in current_oemol.GetAtoms()}
#         new_sidechain_oemol_indices_to_name = {atom.GetIdx(): atom.GetName() for atom in proposed_oemol.GetAtoms()}

#         #now we can get the mol atom map of the sidechain
#         #NOTE: since the sidechain oemols are NOT zero-indexed anymore, we need to match by name (since they are unique identifiers)
#         ring_aas = ['TRP', 'TYR', 'PHE', 'HIS', 'HID', 'HIE', 'HIP']
#         break_bool = False if old_res_name in ring_aas or new_res_name in ring_aas else True
#         _logger.debug(f"\t\t\t allow ring breaking: {break_bool}")
#         # TODO: Refactor to re-use atom_mapper for all ligands?
#         # TODO: Generate atom mapping using only geometries if requested
#         from perses.rjmc.atom_mapping import AtomMapper, InvalidMappingException
#         atom_mapper = AtomMapper(map_strength='strong', matching_criterion='name', allow_ring_breaking=break_bool)
#         atom_mapping = atom_mapper.get_best_mapping(current_oemol, proposed_oemol)

#         #check the atom map thus far:
#         _logger.debug(f"\t\t\tlocal atom map nonstereo sidechain: {atom_mapping}")

#         #preserve chirality of the sidechain
#         # _logger.warning(f"\t\t\told oemols: {[atom.GetIdx() for atom in self.current_molecule.GetAtoms()]}")
#         # _logger.warning(f"\t\t\tnew oemols: {[atom.GetIdx() for atom in new_oemol_res.GetAtoms()]}")
#         # TODO: Should this be done here, or automatically in the mapping generation?
#         if atom_mapping is not None:
#             atom_mapping.preserve_chirality()
#         else:
#             atom_mapping = dict()

#         _logger.debug(f"\t\t\tlocal atom map stereo sidechain: {atom_mapping}")

#         # DEBUG
#         _logger.info(atom_mapping.old_mol.atoms)
#         _logger.info(atom_mapping.new_mol.atoms)

#         #fix the sidechain indices w.r.t. full oemol
#         # TODO: Glue AtomMapping in more broadly
#         sidechain_fixed_map = {}
#         mapped_names = []
#         backbone_atoms = ['N', 'H', 'C', 'O', 'CA', 'HA']
#         sidechain_atom_indices = []
#         for new_sidechain_idx, old_sidechain_idx in atom_mapping.new_to_old_atom_map.items():
#             try:
#                 new_name, old_name = atom_mapping.new_mol.atoms[new_sidechain_idx].name, atom_mapping.old_mol.atoms[old_sidechain_idx].name
#                 mapped_names.append((new_name, old_name))
#                 new_full_oemol_idx, old_full_oemol_idx = new_oemol_name_idx[new_name], old_oemol_name_idx[old_name]
#                 if new_name not in backbone_atoms:
#                     sidechain_atom_indices.append(new_full_oemol_idx)
#                 sidechain_fixed_map[new_full_oemol_idx] = old_full_oemol_idx
#             except KeyError as e:
#                 # Report more information on failure
#                 atom_mapping.render_image('failure.png')
#                 raise Exception(f'{atom_mapping}\n{e}')

#         _logger.debug(f"\t\t\toemol sidechain fixed map: {sidechain_fixed_map}")

#         if ignore_sidechain_atoms:
#             for index in sidechain_atom_indices:
#                 del[sidechain_fixed_map[index]]

#         _logger.debug(f"\t\t\tthe local atom map (backbone) is {local_atom_map}")
#         #update the local map
#         local_atom_map.update(sidechain_fixed_map)
#         _logger.debug(f"\t\t\tthe local atom map (total) is {local_atom_map}")

#         #correct the map
#         #now we have to update the atom map indices
#         _logger.debug(f"\t\t\tadjusting the atom map with topology indices...")
#         topology_index_map = {}
#         for new_oemol_idx, old_oemol_idx in local_atom_map.items():
#             topology_index_map[new_oemol_to_res_map[new_oemol_idx]] = old_oemol_to_res_map[old_oemol_idx]

#         _logger.debug(f"\t\t\ttopology_atom_map: {topology_index_map}")

#         mapped_atoms = [(new_res_index_to_name[new_idx], old_res_index_to_name[old_idx]) for new_idx, old_idx in topology_index_map.items()]
#         print(f"\t\t\tthe mapped atom names are: {mapped_atoms}")

#             #and all of the environment atoms should already be handled
#         return topology_index_map, old_res_to_oemol_map, new_res_to_oemol_map, local_atom_map, current_oemol, proposed_oemol, old_oemol_res_copy, new_oemol_res_copy


In [21]:
class PointMutationExecutor3(PointMutationExecutor):
    """
    Simple, stripped-down class to create a protein-ligand system and allow a mutation of a protein.
    this will allow support for the creation of _two_ relative free energy calculations:
        1. 'wildtype' - 'point mutant' complex hybrid.
        2. 'wildtype' - 'point mutant' protein hybrid (i.e. with ligand of interest unbound)
    Example (create full point mutation executor and run parallel tempering on both complex and apo phases):
        from pkg_resources import resource_filename
        protein_path = 'data/perses_jacs_systems/thrombin/Thrombin_protein.pdb'
        ligands_path = 'data/perses_jacs_systems/thrombin/Thrombin_ligands.sdf'
        protein_filename = resource_filename('openmmforcefields', protein_path)
        ligand_input = resource_filename('openmmforcefields', ligands_path)
        pm_delivery = PointMutationExecutor(protein_filename=protein_filename,
                                    mutation_chain_id='2',
                                    mutation_residue_id='198',
                                     proposed_residue='THR',
                                     phase='complex',
                                     conduct_endstate_validation=False,
                                     ligand_input=ligand_input,
                                     ligand_index=0,
                                     forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml'],
                                     barostat=openmm.MonteCarloBarostat(1.0 * unit.atmosphere, temperature, 50),
                                     forcefield_kwargs={'removeCMMotion': False, 'ewaldErrorTolerance': 1e-4, 'nonbondedMethod': app.PME, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus},
                                     small_molecule_forcefields='gaff-2.11')
        complex_htf = pm_delivery.get_complex_htf()
        apo_htf = pm_delivery.get_apo_htf()
        # Now we can build the hybrid repex samplers
        from perses.annihilation.lambda_protocol import LambdaProtocol
        from openmmtools.multistate import MultiStateReporter
        from perses.samplers.multistate import HybridRepexSampler
        from openmmtools import mcmc
        suffix = 'run'; selection = 'not water'; checkpoint_interval = 10; n_states = 11; n_cycles = 5000
        for htf in [complex_htf, apo_htf]:
            lambda_protocol = LambdaProtocol(functions='default')
            reporter_file = 'reporter.nc'
            reporter = MultiStateReporter(reporter_file, analysis_particle_indices = htf.hybrid_topology.select(selection), checkpoint_interval = checkpoint_interval)
            hss = HybridRepexSampler(mcmc_moves=mcmc.LangevinSplittingDynamicsMove(timestep= 4.0 * unit.femtoseconds,
                                                                                  collision_rate=5.0 / unit.picosecond,
                                                                                  n_steps=250,
                                                                                  reassign_velocities=False,
                                                                                  n_restart_attempts=20,
                                                                                  splitting="V R R R O R R R V",
                                                                                  constraint_tolerance=1e-06),
                                                                                  hybrid_factory=htf,
                                                                                  online_analysis_interval=10,
                                                                                  context_cache=cache.ContextCache(capacity=None, time_to_live=None))
            hss.setup(n_states=n_states, temperature=300*unit.kelvin, storage_file=reporter, lambda_protocol=lambda_protocol, endstates=False)
            hss.extend(n_cycles)
    """
    def __init__(self,
                 protein_filename,
                 mutation_chain_id,
                 mutation_residue_id,
                 proposed_residue,
                 phase='complex',
                 conduct_endstate_validation=True,
                 ligand_input=None,
                 ligand_index=0,
                 allow_undefined_stereo_sdf=False,
                 water_model='tip3p',
                 ionic_strength=0.15 * unit.molar,
                 forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml'],
                 barostat=openmm.MonteCarloBarostat(1.0 * unit.atmosphere, temperature, 50),
                 forcefield_kwargs={'removeCMMotion': False, 'ewaldErrorTolerance': 0.00025, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus},
                 periodic_forcefield_kwargs={'nonbondedMethod': app.PME},
                 nonperiodic_forcefield_kwargs=None,
                 small_molecule_forcefields='gaff-2.11',
                 complex_box_dimensions=None,
                 apo_box_dimensions=None,
                 flatten_torsions=False,
                 flatten_exceptions=False,
                 generate_unmodified_hybrid_topology_factory=True,
                 generate_rest_capable_hybrid_topology_factory=False,
                 **kwargs):
        """
        arguments
            protein_filename : str
                path to protein (to mutate); .pdb
            mutation_chain_id : str
                name of the chain to be mutated
            mutation_residue_id : str
                residue id to change
            proposed_residue : str
                three letter code of the residue to mutate to
            phase : str, default complex
                if phase == vacuum, then the complex will not be solvated with water; else, it will be solvated with tip3p
            conduct_endstate_validation : bool, default True
                whether to conduct an endstate validation of the HybridTopologyFactory. If using the RepartitionedHybridTopologyFactory,
                endstate validation cannot and will not be conducted.
            ligand_input : str, default None
                path to ligand of interest (i.e. small molecule or protein); .sdf or .pdb
            ligand_index : int, default 0
                which ligand to use
            allow_undefined_stereo_sdf : bool, default False
                whether to allow an SDF file to contain undefined stereocenters
            water_model : str, default 'tip3p'
                solvent model to use for solvation
            ionic_strength : float * unit.molar, default 0.15 * unit.molar
                the total concentration of ions (both positive and negative) to add using Modeller.
                This does not include ions that are added to neutralize the system.
                Note that only monovalent ions are currently supported.
            forcefield_files : list of str, default ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
                forcefield files for proteins and solvent
            barostat : openmm.MonteCarloBarostat, default openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
                barostat to use
            forcefield_kwargs : dict, default {'removeCMMotion': False, 'ewaldErrorTolerance': 1e-4, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus}
                forcefield kwargs for system parametrization
            periodic_forcefield_kwargs : dict, default {'nonbondedMethod': app.PME}
                periodic forcefield kwargs for system parametrization
            nonperiodic_forcefield_kwargs : dict, default None
                non-periodic forcefield kwargs for system parametrization
            small_molecule_forcefields : str, default 'gaff-2.11'
                the forcefield string for small molecule parametrization
            complex_box_dimensions : Vec3, default None
                define box dimensions of complex phase;
                if None, padding is 1nm
            apo_box_dimensions :  Vec3, default None
                define box dimensions of apo phase phase;
                if None, padding is 1nm
            flatten_torsions : bool, default False
                in the htf, flatten torsions involving unique new atoms at lambda = 0 and unique old atoms are lambda = 1
            flatten_exceptions : bool, default False
                in the htf, flatten exceptions involving unique new atoms at lambda = 0 and unique old atoms at lambda = 1
            generate_unmodified_hybrid_topology_factory : bool, default True
                whether to generate a vanilla HybridTopologyFactory
            generate_rest_capable_hybrid_topology_factory : bool, default False
                whether to generate a RepartitionedHybridTopologyFactory
        TODO : allow argument for spectator ligands besides the 'ligand_file'
        """
        from openeye import oechem

        # First thing to do is load the apo protein to mutate...
        protein_pdbfile = open(protein_filename, 'r')
        protein_pdb = app.PDBFile(protein_pdbfile)
        protein_pdbfile.close()
        protein_positions, protein_topology, protein_md_topology = protein_pdb.positions, protein_pdb.topology, md.Topology.from_openmm(protein_pdb.topology)
        protein_topology = protein_md_topology.to_openmm()
        protein_n_atoms = protein_md_topology.n_atoms

        # Load the ligand, if present
        molecules = []
        if ligand_input:
            if isinstance(ligand_input, str):
                if ligand_input.endswith('.sdf'): # small molecule
                        ligand_mol = createOEMolFromSDF(ligand_input, index=ligand_index, allow_undefined_stereo=allow_undefined_stereo_sdf)
                        molecules.append(Molecule.from_openeye(ligand_mol, allow_undefined_stereo=False))
                        ligand_positions, ligand_topology = extractPositionsFromOEMol(ligand_mol),  forcefield_generators.generateTopologyFromOEMol(ligand_mol)
                        ligand_md_topology = md.Topology.from_openmm(ligand_topology)
                        ligand_n_atoms = ligand_md_topology.n_atoms

                if ligand_input.endswith('pdb'): # protein
                    ligand_pdbfile = open(ligand_input, 'r')
                    ligand_pdb = app.PDBFile(ligand_pdbfile)
                    ligand_pdbfile.close()
                    ligand_positions, ligand_topology, ligand_md_topology = ligand_pdb.positions, ligand_pdb.topology, md.Topology.from_openmm(
                        ligand_pdb.topology)
                    ligand_n_atoms = ligand_md_topology.n_atoms

            elif isinstance(ligand_input, oechem.OEMol): # oemol object
                molecules.append(Molecule.from_openeye(ligand_input, allow_undefined_stereo=False))
                ligand_positions, ligand_topology = extractPositionsFromOEMol(ligand_input),  forcefield_generators.generateTopologyFromOEMol(ligand_input)
                ligand_md_topology = md.Topology.from_openmm(ligand_topology)
                ligand_n_atoms = ligand_md_topology.n_atoms

            else:
                _logger.warning(f'ligand filetype not recognised. Please provide a path to a .pdb or .sdf file')
                return

            # Now create a complex
            complex_md_topology = protein_md_topology.join(ligand_md_topology)
            complex_topology = complex_md_topology.to_openmm()
            complex_positions = unit.Quantity(np.zeros([protein_n_atoms + ligand_n_atoms, 3]), unit=unit.nanometers)
            complex_positions[:protein_n_atoms, :] = protein_positions
            complex_positions[protein_n_atoms:, :] = ligand_positions

            # Convert positions back to openmm vec3 objects
            complex_positions_vec3 = []
            for position in complex_positions:
                complex_positions_vec3.append(openmm.Vec3(*position.value_in_unit_system(unit.md_unit_system)))
            complex_positions = unit.Quantity(value=complex_positions_vec3, unit=unit.nanometer)

        # Now for a system_generator
        self.system_generator = SystemGenerator(forcefields=forcefield_files,
                                                barostat=barostat,
                                                forcefield_kwargs=forcefield_kwargs,
                                                periodic_forcefield_kwargs=periodic_forcefield_kwargs,
                                                nonperiodic_forcefield_kwargs=nonperiodic_forcefield_kwargs,
                                                small_molecule_forcefield=small_molecule_forcefields,
                                                molecules=molecules,
                                                cache=None)

        # Solvate apo and complex...
        apo_input = list(self._solvate(protein_topology, protein_positions, water_model, phase, ionic_strength, apo_box_dimensions))
        inputs = [apo_input]
        if ligand_input:
            inputs.append(self._solvate(complex_topology, complex_positions, water_model, phase, ionic_strength, complex_box_dimensions))

        geometry_engine = FFAllAngleGeometryEngine(metadata=None,
                                                use_sterics=False,
                                                n_bond_divisions=100,
                                                n_angle_divisions=180,
                                                n_torsion_divisions=360,
                                                verbose=True,
                                                storage=None,
                                                bond_softening_constant=1.0,
                                                angle_softening_constant=1.0,
                                                neglect_angles = False,
                                                use_14_nonbondeds = True)


        # Run pipeline...
        htfs = []
        for is_complex, (top, pos, sys) in enumerate(inputs):
            if not is_complex:
                point_mutation_engine = PointMutationEngine2(wildtype_topology=top,
                                                                     system_generator=self.system_generator,
                                                                     chain_id=mutation_chain_id, # Denote the chain id allowed to mutate (it's always a string variable)
                                                                     max_point_mutants=1,
                                                                     residues_allowed_to_mutate=[mutation_residue_id], # The residue ids allowed to mutate
                                                                     allowed_mutations=[(mutation_residue_id, proposed_residue)], # The residue ids allowed to mutate with the three-letter code allowed to change
                                                                     aggregate=True) # Always allow aggregation

                topology_proposal = point_mutation_engine.propose(sys, top)
                self.topology_proposal = topology_proposal
                
                # Only validate energy bookkeeping if the WT and proposed residues do not involve rings
                old_res = [res for res in top.residues() if res.id == mutation_residue_id][0]
                validate_bool = False if old_res.name in ring_amino_acids or proposed_residue in ring_amino_acids else True
                new_positions, logp_proposal = geometry_engine.propose(topology_proposal, pos, beta,
                                                                       validate_energy_bookkeeping=validate_bool)
                logp_reverse = geometry_engine.logp_reverse(topology_proposal, new_positions, pos, beta,
                                                        validate_energy_bookkeeping=validate_bool)
                self.new_positions = new_positions
    

In [22]:
solvent_delivery = PointMutationExecutor3("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/input/asn_vacuum.pdb",
                        '1',
                        "2",
                        "GLN",
                        forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml'],
                        ionic_strength=0.15*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                        )

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:setup:solvating at 0.15 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:local_atom_map: {6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14}
INFO:proposal_generator:the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('CB', 'CB'), ('HB2', 'HB2'), ('HB3', 'HB3'), ('C', 'C'), ('O', 'O')]


HERE's the atom map from construct_atom_map() {6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14}


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 8
INFO:geometry:Atom index proposal order is [15, 18, 19, 20, 21, 16, 17, 22]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 13 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 48 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 76 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching fun

conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced potential before atom placement: 22.94759492094766


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: -113.87283657041934
INFO:geometry:final reduced energy -90.92524967369573
INFO:geometry:sum of energies: -90.92524164947167
INFO:geometry:magnitude of difference in the energies: 8.02422405854486e-06
INFO:geometry:Final logp_proposal: -15591.346619502525


added energy components: [('CustomBondForce', 0.8000874729196442), ('CustomAngleForce', 4.478894460315938), ('CustomTorsionForce', 17.351352121183233), ('CustomBondForce', -136.50317062483813)]


In [33]:
solvent_delivery = PointMutationExecutor3("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/input/asn_vacuum.pdb",
                        '1',
                        "2",
                        "THR",
                        forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml'],
                        ionic_strength=0.15*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                        )

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:setup:solvating at 0.15 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:local_atom_map: {6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 13: 12, 11: 13, 12: 14}
INFO:proposal_generator:the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('CB', 'CB'), ('HB', 'HB3'), ('C', 'C'), ('O', 'O')]


HERE's the atom map from construct_atom_map() {6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 13: 12, 11: 13, 12: 14}


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 6
INFO:geometry:Atom index proposal order is [18, 14, 19, 16, 17, 15]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 11 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 43 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 72 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, s

conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced potential before atom placement: 21.66829850589605


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: -107.33023103083794
INFO:geometry:final reduced energy -85.66194071081902
INFO:geometry:sum of energies: -85.66193252494189
INFO:geometry:magnitude of difference in the energies: 8.18587713524721e-06
INFO:geometry:Final logp_proposal: -15495.504678599173


added energy components: [('CustomBondForce', 0.8000874729196442), ('CustomAngleForce', 5.472506641416873), ('CustomTorsionForce', 17.637036325715922), ('CustomBondForce', -131.23986147089042)]


In [14]:
solvent_delivery = PointMutationExecutor3("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/input/rbd_protonated.pdb",
                        '1',
                        "501",
                        "GLN",
                        forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml', '/home/zhangi/choderalab/openmmforcefields/amber/ffxml/GLYCAM_06j-1.xml'],
                        ionic_strength=0.15*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                        )

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:setup:solvating at 0.15 M using tip3p


KeyboardInterrupt: 

In [25]:
solvent_delivery = PointMutationExecutor3("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/input/rbd_protonated.pdb",
                        '1',
                        "501",
                        "GLY",
                        forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml', '/home/zhangi/choderalab/openmmforcefields/amber/ffxml/GLYCAM_06j-1.xml'],
                        ionic_strength=0.15*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                        )

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:setup:solvating at 0.15 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...


LOCAL ATOM MAP:  {2605: 2605, 2606: 2606, 2607: 2607, 2608: 2609, 2609: 2610}
			the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('C', 'C'), ('O', 'O')]
HERE's the atom map from construct_atom_map() {2605: 2605, 2606: 2606, 2607: 2607, 2608: 2609, 2609: 2610}
ALL OTHER ATOMS MAP:  {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17, 18: 18, 19: 19, 20: 20, 21: 21, 22: 22, 23: 23, 24: 24, 25: 25, 26: 26, 27: 27, 28: 28, 29: 29, 30: 30, 31: 31, 32: 32, 33: 33, 34: 34, 35: 35, 36: 36, 37: 37, 38: 38, 39: 39, 40: 40, 41: 41, 42: 42, 43: 43, 44: 44, 45: 45, 46: 46, 47: 47, 48: 48, 49: 49, 50: 50, 51: 51, 52: 52, 53: 53, 54: 54, 55: 55, 56: 56, 57: 57, 58: 58, 59: 59, 60: 60, 61: 61, 62: 62, 63: 63, 64: 64, 65: 65, 66: 66, 67: 67, 68: 68, 69: 69, 70: 70, 71: 71, 72: 72, 73: 73, 74: 74, 75: 75, 76: 76, 77: 77, 78: 78, 79: 79, 80: 80, 81: 81, 82: 82, 83: 83, 84: 84, 85: 85, 86: 86, 87: 87, 88: 88, 89: 89,

NameError: name 'SmallMoleculeSetProposalEngine' is not defined

In [27]:
solvent_delivery = PointMutationExecutor3("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/input/rbd_protonated.pdb",
                        '1',
                        "501",
                        "ARG",
                        forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml', '/home/zhangi/choderalab/openmmforcefields/amber/ffxml/GLYCAM_06j-1.xml'],
                        ionic_strength=0.15*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                        )

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:setup:solvating at 0.15 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...


LOCAL ATOM MAP:  {2605: 2605, 2606: 2606, 2607: 2607, 2608: 2608, 2609: 2609, 2610: 2610, 2611: 2611, 2612: 2612, 2613: 2613}
			the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('C', 'C'), ('O', 'O'), ('CB', 'CB'), ('HB2', 'HB2'), ('HB3', 'HB3')]
HERE's the atom map from construct_atom_map() {2605: 2605, 2606: 2606, 2607: 2607, 2608: 2608, 2609: 2609, 2610: 2610, 2611: 2611, 2612: 2612, 2613: 2613}


NameError: name 'SmallMoleculeSetProposalEngine' is not defined

In [28]:
solvent_delivery = PointMutationExecutor3("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/input/rbd_protonated.pdb",
                        '1',
                        "501",
                        "TYR",
                        forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml', '/home/zhangi/choderalab/openmmforcefields/amber/ffxml/GLYCAM_06j-1.xml'],
                        ionic_strength=0.15*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                        )

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:setup:solvating at 0.15 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...


LOCAL ATOM MAP:  {2605: 2605, 2606: 2606, 2607: 2607, 2608: 2608, 2609: 2609, 2610: 2610, 2611: 2611, 2612: 2612, 2613: 2613}
			the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('C', 'C'), ('O', 'O'), ('CB', 'CB'), ('HB2', 'HB2'), ('HB3', 'HB3')]
HERE's the atom map from construct_atom_map() {2605: 2605, 2606: 2606, 2607: 2607, 2608: 2608, 2609: 2609, 2610: 2610, 2611: 2611, 2612: 2612, 2613: 2613}


NameError: name 'SmallMoleculeSetProposalEngine' is not defined

In [29]:
solvent_delivery = PointMutationExecutor3("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/input/rbd_protonated.pdb",
                        '1',
                        "501",
                        "ALA",
                        forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml', '/home/zhangi/choderalab/openmmforcefields/amber/ffxml/GLYCAM_06j-1.xml'],
                        ionic_strength=0.15*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                        )

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:setup:solvating at 0.15 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...


LOCAL ATOM MAP:  {2605: 2605, 2606: 2606, 2607: 2607, 2608: 2608, 2609: 2609, 2610: 2610, 2611: 2611, 2612: 2612, 2613: 2613}
			the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('C', 'C'), ('O', 'O'), ('CB', 'CB'), ('HB2', 'HB2'), ('HB3', 'HB3')]
HERE's the atom map from construct_atom_map() {2605: 2605, 2606: 2606, 2607: 2607, 2608: 2608, 2609: 2609, 2610: 2610, 2611: 2611, 2612: 2612, 2613: 2613}


NameError: name 'SmallMoleculeSetProposalEngine' is not defined

In [30]:
solvent_delivery = PointMutationExecutor3("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/input/rbd_protonated.pdb",
                        '1',
                        "501",
                        "VAL",
                        forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml', '/home/zhangi/choderalab/openmmforcefields/amber/ffxml/GLYCAM_06j-1.xml'],
                        ionic_strength=0.15*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                        )

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:setup:solvating at 0.15 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...


LOCAL ATOM MAP:  {2605: 2605, 2606: 2606, 2607: 2607, 2608: 2608, 2609: 2609, 2610: 2610, 2612: 2612}
			the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('C', 'C'), ('O', 'O'), ('HB', 'HB2')]
HERE's the atom map from construct_atom_map() {2605: 2605, 2606: 2606, 2607: 2607, 2608: 2608, 2609: 2609, 2610: 2610, 2612: 2612}


NameError: name 'SmallMoleculeSetProposalEngine' is not defined

In [31]:
solvent_delivery = PointMutationExecutor3("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/input/rbd_protonated.pdb",
                        '1',
                        "503", # VAL
                        "ILE",
                        forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml', '/home/zhangi/choderalab/openmmforcefields/amber/ffxml/GLYCAM_06j-1.xml'],
                        ionic_strength=0.15*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                        )

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:setup:solvating at 0.15 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...


LOCAL ATOM MAP:  {2626: 2626, 2627: 2627, 2628: 2628, 2629: 2629, 2630: 2630, 2631: 2631, 2632: 2632, 2633: 2633}
			the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('C', 'C'), ('O', 'O'), ('CB', 'CB'), ('HB', 'HB')]
HERE's the atom map from construct_atom_map() {2626: 2626, 2627: 2627, 2628: 2628, 2629: 2629, 2630: 2630, 2631: 2631, 2632: 2632, 2633: 2633}


NameError: name 'SmallMoleculeSetProposalEngine' is not defined

In [ ]:
solvent_delivery = PointMutationExecutor3("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/input/rbd_protonated.pdb",
                        '1',
                        "522", # ALA
                        "VAL",
                        forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml', '/home/zhangi/choderalab/openmmforcefields/amber/ffxml/GLYCAM_06j-1.xml'],
                        ionic_strength=0.15*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                        )

In [33]:
solvent_delivery = PointMutationExecutor3("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/input/rbd_protonated.pdb",
                        '1',
                        "522", # ALA
                        "TYR",
                        forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml', '/home/zhangi/choderalab/openmmforcefields/amber/ffxml/GLYCAM_06j-1.xml'],
                        ionic_strength=0.15*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                        )

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:setup:solvating at 0.15 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...


LOCAL ATOM MAP:  {2938: 2938, 2939: 2939, 2940: 2940, 2941: 2941, 2942: 2942, 2943: 2943, 2944: 2944, 2945: 2946, 2946: 2947}
			the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('C', 'C'), ('O', 'O'), ('CB', 'CB'), ('HB2', 'HB2'), ('HB3', 'HB3')]
HERE's the atom map from construct_atom_map() {2938: 2938, 2939: 2939, 2940: 2940, 2941: 2941, 2942: 2942, 2943: 2943, 2944: 2944, 2945: 2946, 2946: 2947}


NameError: name 'SmallMoleculeSetProposalEngine' is not defined

In [58]:
solvent_delivery = PointMutationExecutor3("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/input/rbd_protonated.pdb",
                        '1',
                        "522", # ALA
                        "GLY",
                        forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml', '/home/zhangi/choderalab/openmmforcefields/amber/ffxml/GLYCAM_06j-1.xml'],
                        ionic_strength=0.15*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                        )

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:setup:solvating at 0.15 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:local_atom_map: {2938: 2938, 2939: 2939, 2940: 2940, 2941: 2942, 2942: 2943}
INFO:proposal_generator:the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('C', 'C'), ('O', 'O')]


HERE's the atom map from construct_atom_map() {2938: 2938, 2939: 2939, 2940: 2940, 2941: 2942, 2942: 2943}


NameError: name 'SmallMoleculeSetProposalEngine' is not defined

# Check to see if ASN to GLN is messed up for dipeptides too

In [42]:
solvent_delivery = PointMutationExecutor3("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/input/asn_vacuum.pdb",
                        '1',
                        "2", 
                        "GLN",
                        forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml'],
                        ionic_strength=0.15*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                        )

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:setup:solvating at 0.15 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG, atomic number=6), Atom(name=ND2, atomic number=7), Atom(name=OD1, atomic number=8), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1), Atom(name=HD21, atomic number=1), Atom(name=HD22, atomic number=1)]
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG, atomic number=6), Atom(name=CD, atomic number=6), Atom(name=NE2, atomic number=7), Atom(name=OE1, atomic number=8), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1), Atom(name=HG2, atomic number=1), Atom(name=HG3, atomic number=1), Atom(name=HE21, atomic number=1), Atom(name=HE22, atomic number=1)]


			the mapped atom names are: [('N', 'N'), ('CA', 'CA'), ('C', 'C'), ('O', 'O'), ('H', 'H'), ('HA', 'HA'), ('CD', 'CG'), ('NE2', 'ND2'), ('OE1', 'OD1'), ('HE22', 'HD21'), ('HE21', 'HD22')]
HERE's the atom map from construct_atom_map() {6: 6, 8: 8, 13: 13, 14: 14, 7: 7, 9: 9, 18: 15, 20: 17, 19: 16, 22: 18, 21: 19}


NameError: name 'SmallMoleculeSetProposalEngine' is not defined

In [43]:
solvent_delivery = PointMutationExecutor3("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/input/asn_vacuum.pdb",
                        '1',
                        "2", 
                        "ARG",
                        forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml'],
                        ionic_strength=0.15*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                        )

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:setup:solvating at 0.15 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG, atomic number=6), Atom(name=ND2, atomic number=7), Atom(name=OD1, atomic number=8), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1), Atom(name=HD21, atomic number=1), Atom(name=HD22, atomic number=1)]
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG, atomic number=6), Atom(name=CD, atomic number=6), Atom(name=NE, atomic number=7), Atom(name=CZ, atomic number=6), Atom(name=NH1, atomic number=7), Atom(name=NH2, atomic number=7), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1), Atom(name=HG2, atomic number=1), Atom(name=HG3, atomic number=1), Atom(name=HD2, atomic number=1), Atom(name=HD3, atomic number=1), Atom(name=HE, atomic number=1), Atom(name=HH11, atomic numb

			the mapped atom names are: [('N', 'N'), ('CA', 'CA'), ('C', 'C'), ('O', 'O'), ('H', 'H'), ('HA', 'HA'), ('CZ', 'CG'), ('NH1', 'ND2'), ('HH12', 'HD21'), ('HH11', 'HD22')]
HERE's the atom map from construct_atom_map() {6: 6, 8: 8, 13: 13, 14: 14, 7: 7, 9: 9, 23: 15, 24: 17, 26: 18, 25: 19}


NameError: name 'SmallMoleculeSetProposalEngine' is not defined

In [44]:
solvent_delivery = PointMutationExecutor3("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/input/asn_vacuum.pdb",
                        '1',
                        "2", 
                        "GLY",
                        forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml'],
                        ionic_strength=0.15*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                        )

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:setup:solvating at 0.15 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG, atomic number=6), Atom(name=ND2, atomic number=7), Atom(name=OD1, atomic number=8), Atom(name=HA, atomic number=1), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1), Atom(name=HD21, atomic number=1), Atom(name=HD22, atomic number=1)]
INFO:proposal_generator:[Atom(name=HA3, atomic number=1), Atom(name=HA2, atomic number=1)]


			the mapped atom names are: [('N', 'N'), ('CA', 'CA'), ('C', 'C'), ('O', 'O'), ('H', 'H'), ('HA3', 'HD21')]
HERE's the atom map from construct_atom_map() {6: 6, 8: 8, 9: 13, 10: 14, 7: 7, 12: 18}


NameError: name 'SmallMoleculeSetProposalEngine' is not defined

# Write tests

In [ ]:
def test_protein_atom_maps():
    
    # Get alanine dipeptide in vacuum test system
    ala, system_generator = generate_atp()
    
    
    # Define function for checking that the atom map is correct
    def check_atom_map(topology_proposal, reference_map):
        # Retrieve atom index to name mapping for old and new residues
        old_res = [res for res in topology_proposal.old_topology.residues() if res.name == topology_proposal.old_residue_name][0]
        new_res = [res for res in topology_proposal.new_topology.residues() if res.name == topology_proposal.new_residue_name][0]
        old_res_index_to_name = {atom.index: atom.name for atom in old_res.atoms()}
        new_res_index_to_name = {atom.index: atom.name for atom in new_res.atoms()}
        
        # Check whether the atom map generated matches the reference map
        atom_map = topology_proposal._core_new_to_old_atom_map
        mapped_atoms = [(new_res_index_to_name[new_idx], old_res_index_to_name[old_idx]) for new_idx, old_idx in atom_map.items()]
        assert reference_map == mapped_atoms, f"{topology_proposal.old_residue_name}->{topology_proposal.new_residue_name} map does not match reference map"
    
    # ALA -> TYR
    topology_proposal, new_positions, logp_proposal, logp_reverse = generate_dipeptide_top_pos_sys(ala.topology, 
                                                                                                           'TYR', 
                                                                                                           ala.system, 
                                                                                                           ala.positions, 
                                                                                                           system_generator, 
                                                                                                           conduct_geometry_prop=True)
    tyr_topology, tyr_system = topology_proposal.new_topology, topology_proposal.new_system
    reference_map = [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('C', 'C'), ('O', 'O'), ('CB', 'CB'), ('HB2', 'HB2'), ('HB3', 'HB3')]
    check_atom_map(topology_proposal, reference_map)
    
    # TYR -> ALA
    topology_proposal, new_positions, logp_proposal, logp_reverse = generate_dipeptide_top_pos_sys(tyr_topology, 
                                                                                                       'ALA', 
                                                                                                       tyr_system, 
                                                                                                       new_positions, 
                                                                                                       system_generator, 
                                                                                                       conduct_geometry_prop=True)
    reference_map = [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('C', 'C'), ('O', 'O'), ('CB', 'CB'), ('HB2', 'HB2'), ('HB3', 'HB3')]
    check_atom_map(topology_proposal, reference_map)
    
    # ALA -> VAL
    topology_proposal, new_positions, logp_proposal, logp_reverse = generate_dipeptide_top_pos_sys(ala.topology, 
                                                                                                           'VAL', 
                                                                                                           ala.system, 
                                                                                                           ala.positions, 
                                                                                                           system_generator, 
                                                                                                           conduct_geometry_prop=True)
    val_topology, val_system = topology_proposal.new_topology, topology_proposal.new_system
    reference_map = [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('C', 'C'), ('O', 'O'), ('HB', 'HB2')]
    check_atom_map(topology_proposal, reference_map)
    
    # VAL -> ALA
    topology_proposal, new_positions, logp_proposal, logp_reverse = generate_dipeptide_top_pos_sys(val_topology, 
                                                                                                       'ALA', 
                                                                                                       val_system, 
                                                                                                       val_positions, 
                                                                                                       system_generator, 
                                                                                                       conduct_geometry_prop=True)
    reference_map = [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('C', 'C'), ('O', 'O'), ('HB2', 'HB')]
    check_atom_map(topology_proposal, reference_map)
    
    # VAL -> ILE
    topology_proposal, new_positions, logp_proposal, logp_reverse = generate_dipeptide_top_pos_sys(val_topology, 
                                                                                                       'ILE', 
                                                                                                       val_system, 
                                                                                                       val_positions, 
                                                                                                       system_generator, 
                                                                                                       conduct_geometry_prop=True)
    reference_map = [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('C', 'C'), ('O', 'O'), ('CB', 'CB'), ('HB', 'HB')]
    check_atom_map(topology_proposal, reference_map)
    
    # ALA -> GLY
    topology_proposal, new_positions, logp_proposal, logp_reverse = generate_dipeptide_top_pos_sys(ala.topology, 
                                                                                                           'GLY', 
                                                                                                           ala.system, 
                                                                                                           ala.positions, 
                                                                                                           system_generator, 
                                                                                                          conduct_geometry_prop=True)
    gly_topology, gly_system = topology_proposal.new_topology, topology_proposal.new_system
    reference_map = [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('C', 'C'), ('O', 'O')]
    check_atom_map(topology_proposal, reference_map)
    
    # GLY -> ALA
    topology_proposal, new_positions, logp_proposal, logp_reverse = generate_dipeptide_top_pos_sys(gly_topology, 
                                                                                                   'ALA', 
                                                                                                   gly_system, 
                                                                                                   gly_positions, 
                                                                                                   system_generator, 
                                                                                                   conduct_geometry_prop=True)
    reference_map = [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('C', 'C'), ('O', 'O')]
    check_atom_map(topology_proposal, reference_map)
    
    # Check that the 

In [3]:
from perses.tests.test_topology_proposal import generate_dipeptide_top_pos_sys, generate_atp

In [5]:
# Get alanine dipeptide in vacuum test system
ala, system_generator = generate_atp()
    
    
# ALA -> TYR
topology_proposal, new_positions, logp_proposal, logp_reverse = generate_dipeptide_top_pos_sys(ala.topology, 
                                                                                                       'THR', 
                                                                                                       ala.system, 
                                                                                                       ala.positions, 
                                                                                                       system_generator, 
                                                                                                       conduct_geometry_prop=True)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=HB1, atomic number=1), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1)]
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG2, atomic number=6), Atom(name=OG1, atomic number=8), Atom(name=HB, atomic number=1), Atom(name=HG1, atomic number=1), Atom(name=HG21, atomic number=1), Atom(name=HG22, atomic number=1), Atom(name=HG23, atomic number=1)]
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 6
INFO:geometry:Atom index proposal order is [14, 18, 17, 15, 16, 19]
INFO:geometry:omitted_bonds: []
INFO:geometry:d

making topology proposal
generating geometry engine
making geometry proposal from ALA to THR
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 6 new atoms
INFO:geometry:	reduced angle potential = 0.06691548111536394.
INFO:geometry:	reduced angle potential = 0.4525176858551157.
INFO:geometry:	reduced angle potential = 0.28356981549975396.
INFO:geometry:	reduced angle potential = 0.017175461530447113.
INFO:geometry:	reduced angle potential = 3.112611707072305.
INFO:geometry:	reduced angle potential = 0.15259273506325846.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 11 bond forces in the no-nonbonded final system
INFO:geometry:	there are 43 angle forces in the no-nonbonded final system
INFO:geometry:	there are 72 torsion forces in the no-nonbonded final system
INFO:geo

...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced potential before atom placement: 16.81551376691254


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 7.104923552150649
INFO:geometry:final reduced energy 23.92043716104155
INFO:geometry:sum of energies: 23.92043731906319
INFO:geometry:magnitude of difference in the energies: 1.580216393293199e-07
INFO:geometry:Final logp_proposal: -17809.925873232816


added energy components: [('CustomBondForce', 0.0), ('CustomAngleForce', 0.0001781007424788738), ('CustomTorsionForce', 0.0028912674803934445), ('CustomBondForce', 7.101854183927775)]


In [23]:
import simtk.openmm.app as app
import simtk.openmm as openmm
import simtk.unit as unit
from pkg_resources import resource_filename
import numpy as np
import os
try:
    from urllib.request import urlopen
    from io import StringIO
except:
    from urllib2 import urlopen
    from cStringIO import StringIO
from nose.plugins.attrib import attr

from openmmtools.constants import kB
from perses.utils.openeye import OEMol_to_omm_ff, smiles_to_oemol
from perses.utils.smallmolecules import render_atom_mapping
from perses.rjmc.topology_proposal import SmallMoleculeSetProposalEngine
from perses.rjmc import topology_proposal
from collections import defaultdict
from openmmforcefields.generators import SystemGenerator
from openff.toolkit.topology import Molecule
from openmoltools.forcefield_generators import generateOEMolFromTopologyResidue

#default arguments for SystemGenerators
barostat = None
forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
forcefield_kwargs = {'removeCMMotion': False, 'ewaldErrorTolerance': 1e-4, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus}
nonperiodic_forcefield_kwargs = {'nonbondedMethod': app.NoCutoff}
small_molecule_forcefield = 'gaff-2.11'

temperature = 300*unit.kelvin
# Compute kT and inverse temperature.
kT = kB * temperature
beta = 1.0 / kT
ENERGY_THRESHOLD = 1e-6
PROHIBITED_RESIDUES = ['CYS']

In [25]:
def generate_dipeptide_top_pos_sys2(topology,
                                   new_res,
                                   system,
                                   positions,
                                   system_generator,
                                   conduct_geometry_prop = True,
                                   conduct_htf_prop = False,
                                   validate_energy_bookkeeping=True,
                                   repartitioned=False,
                                   endstate=None,
                                   flatten_torsions=False,
                                   flatten_exceptions=False,
                                   validate_endstate_energy=True
                                   ):
    """generate point mutation engine, geometry_engine, and conduct topology proposal, geometry propsal, and hybrid factory generation"""
    from perses.tests.utils import validate_endstate_energies
    if conduct_htf_prop:
        assert conduct_geometry_prop, f"the htf prop can only be conducted if there is a geometry proposal"
    # Create the point mutation engine
    from perses.rjmc.topology_proposal import PointMutationEngine
    point_mutation_engine = PointMutationEngine2(wildtype_topology=topology,
                                                system_generator=system_generator,
                                                chain_id='1', # Denote the chain id allowed to mutate (it's always a string variable)
                                                max_point_mutants=1,
                                                residues_allowed_to_mutate=['2'], # The residue ids allowed to mutate
                                                allowed_mutations=[('2', new_res)], # The residue ids allowed to mutate with the three-letter code allowed to change
                                                aggregate=True) # Always allow aggregation

    # Create a top proposal
    print(f"making topology proposal")
    topology_proposal = point_mutation_engine.propose(current_system=system, current_topology=topology)

    if not conduct_geometry_prop:
        return topology_proposal

    if conduct_geometry_prop:
        # Create a geometry engine
        print(f"generating geometry engine")
        from perses.rjmc.geometry import FFAllAngleGeometryEngine
        geometry_engine = FFAllAngleGeometryEngine(metadata=None,
                                               use_sterics=False,
                                               n_bond_divisions=100,
                                               n_angle_divisions=180,
                                               n_torsion_divisions=360,
                                               verbose=True,
                                               storage=None,
                                               bond_softening_constant=1.0,
                                               angle_softening_constant=1.0,
                                               neglect_angles = False,
                                               use_14_nonbondeds = True)


        # Make a geometry proposal forward
        print(f"making geometry proposal from {list(topology.residues())[1].name} to {new_res}")
        forward_new_positions, logp_proposal = geometry_engine.propose(topology_proposal, positions, beta, validate_energy_bookkeeping=validate_energy_bookkeeping)
        logp_reverse = geometry_engine.logp_reverse(topology_proposal, forward_new_positions, positions, beta, validate_energy_bookkeeping=validate_energy_bookkeeping)

    if not conduct_htf_prop:
        return (topology_proposal, forward_new_positions, logp_proposal, logp_reverse)

In [36]:
# Get alanine dipeptide in vacuum test system
ala, system_generator = generate_atp()
    
    
# ALA -> VAL
topology_proposal, new_positions, logp_proposal, logp_reverse = generate_dipeptide_top_pos_sys(ala.topology, 
                                                                                                       'VAL', 
                                                                                                       ala.system, 
                                                                                                       ala.positions, 
                                                                                                       system_generator, 
                                                                                                       conduct_geometry_prop=True)
val_topology, val_system = topology_proposal.new_topology, topology_proposal.new_system
reference_map = [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('C', 'C'), ('O', 'O'), ('HB', 'HB2')]

# VAL -> ILE
topology_proposal, new_positions, logp_proposal, logp_reverse = generate_dipeptide_top_pos_sys(val_topology, 
                                                                                                   'ILE', 
                                                                                                   val_system, 
                                                                                                   new_positions, 
                                                                                                   system_generator, 
                                                                                                   conduct_geometry_prop=True)
# reference_map = [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('C', 'C'), ('O', 'O'), ('CB', 'CB'), ('HB', 'HB')]
# check_atom_map(topology_proposal, reference_map)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:local_atom_map: {6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 13: 13, 11: 14, 12: 15}
INFO:proposal_generator:the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('CB', 'CB'), ('HB', 'HB3'), ('C', 'C'), ('O', 'O')]
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 8
INFO:geometry:Atom index proposal order is [14, 18, 16, 21, 15, 19, 20, 17]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force.

making topology proposal
HERE's the atom map from construct_atom_map() {6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 13: 13, 11: 14, 12: 15}
generating geometry engine
making geometry proposal from ALA to VAL
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 8 new atoms
INFO:geometry:	reduced angle potential = 0.0013020865823623522.
INFO:geometry:	reduced angle potential = 0.0012742850203132144.
INFO:geometry:	reduced angle potential = 0.016928610005650803.
INFO:geometry:	reduced angle potential = 0.023833394951472475.
INFO:geometry:	reduced angle potential = 0.007305358666147016.
INFO:geometry:	reduced angle potential = 0.20459251163077452.
INFO:geometry:	reduced angle potential = 1.693785207333494.
INFO:geometry:	reduced angle potential = 1.0864369966797778.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 11 bond forces in the no-nonbonded final system
INFO:geometry:	there are 48 

...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced potential before atom placement: 16.81551376691254


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: -44.13223278215812
INFO:geometry:final reduced energy -27.316721737222245
INFO:geometry:sum of energies: -27.316719015245578
INFO:geometry:magnitude of difference in the energies: 2.721976663622172e-06
INFO:geometry:Final logp_proposal: 49.85404427988245


added energy components: [('CustomBondForce', 0.46245122195612925), ('CustomAngleForce', 5.466980985219015), ('CustomTorsionForce', 8.384786246764472), ('CustomBondForce', -58.44645123609775)]


In [39]:
for atom in topology_proposal.old_topology.atoms():
    print(atom)

<Atom 0 (H1) of chain 0 residue 0 (ACE)>
<Atom 1 (CH3) of chain 0 residue 0 (ACE)>
<Atom 2 (H2) of chain 0 residue 0 (ACE)>
<Atom 3 (H3) of chain 0 residue 0 (ACE)>
<Atom 4 (C) of chain 0 residue 0 (ACE)>
<Atom 5 (O) of chain 0 residue 0 (ACE)>
<Atom 6 (N) of chain 0 residue 1 (VAL)>
<Atom 7 (H) of chain 0 residue 1 (VAL)>
<Atom 8 (CA) of chain 0 residue 1 (VAL)>
<Atom 9 (HA) of chain 0 residue 1 (VAL)>
<Atom 10 (CB) of chain 0 residue 1 (VAL)>
<Atom 11 (C) of chain 0 residue 1 (VAL)>
<Atom 12 (O) of chain 0 residue 1 (VAL)>
<Atom 13 (HB) of chain 0 residue 1 (VAL)>
<Atom 14 (CG1) of chain 0 residue 1 (VAL)>
<Atom 15 (HG11) of chain 0 residue 1 (VAL)>
<Atom 16 (HG12) of chain 0 residue 1 (VAL)>
<Atom 17 (HG13) of chain 0 residue 1 (VAL)>
<Atom 18 (CG2) of chain 0 residue 1 (VAL)>
<Atom 19 (HG21) of chain 0 residue 1 (VAL)>
<Atom 20 (HG22) of chain 0 residue 1 (VAL)>
<Atom 21 (HG23) of chain 0 residue 1 (VAL)>
<Atom 22 (N) of chain 0 residue 2 (NME)>
<Atom 23 (H) of chain 0 residue 2 (N

In [40]:
for atom in topology_proposal.new_topology.atoms():
    print(atom)

<Atom 0 (H1) of chain 0 residue 0 (ACE)>
<Atom 1 (CH3) of chain 0 residue 0 (ACE)>
<Atom 2 (H2) of chain 0 residue 0 (ACE)>
<Atom 3 (H3) of chain 0 residue 0 (ACE)>
<Atom 4 (C) of chain 0 residue 0 (ACE)>
<Atom 5 (O) of chain 0 residue 0 (ACE)>
<Atom 6 (N) of chain 0 residue 1 (ILE)>
<Atom 7 (H) of chain 0 residue 1 (ILE)>
<Atom 8 (CA) of chain 0 residue 1 (ILE)>
<Atom 9 (HA) of chain 0 residue 1 (ILE)>
<Atom 10 (CB) of chain 0 residue 1 (ILE)>
<Atom 11 (C) of chain 0 residue 1 (ILE)>
<Atom 12 (O) of chain 0 residue 1 (ILE)>
<Atom 13 (HB) of chain 0 residue 1 (ILE)>
<Atom 14 (CG1) of chain 0 residue 1 (ILE)>
<Atom 15 (HG12) of chain 0 residue 1 (ILE)>
<Atom 16 (HG13) of chain 0 residue 1 (ILE)>
<Atom 17 (CG2) of chain 0 residue 1 (ILE)>
<Atom 18 (HG21) of chain 0 residue 1 (ILE)>
<Atom 19 (HG22) of chain 0 residue 1 (ILE)>
<Atom 20 (HG23) of chain 0 residue 1 (ILE)>
<Atom 21 (CD1) of chain 0 residue 1 (ILE)>
<Atom 22 (HD11) of chain 0 residue 1 (ILE)>
<Atom 23 (HD12) of chain 0 residue

In [1]:
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys
from simtk.openmm import app

DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=6)
           2	LOAD_GLOBAL(arg=0, lineno=6)
           4	LOAD_METHOD(arg=1, lineno=6)
           6	LOAD_CONST(arg=1, lineno=6)
           8	CALL_METHOD(arg=1, lineno=6)
          10	STORE_FAST(arg=2, lineno=6)
          12	LOAD_FAST(arg=0, lineno=7)
          14	LOAD_CONST(arg=2, lineno=7)
          16	BINARY_SUBSCR(arg=None, lineno=7)
          18	LOAD_FAST(arg=1, lineno=7)
          20	LOAD_CONST(arg=3, lineno=7)
          22	BINARY_SUBSCR(arg=None, lineno=7)
          24	BINARY_MULTIPLY(arg=None, lineno=7)
          26	LOAD_FAST(arg=0, lineno=7)
          28	LOAD_CONST(arg=3, lineno=7)
          30	BINARY_SUBSCR(arg=None, lineno=7)
          32	LOAD_FAST(arg=1, lineno=7)
          34	LOAD_CONST(arg=2, lineno=7)
          36	BINARY_SUBSCR(arg=None, lineno=7)
          38	BINARY_MULTIPLY(arg=None, lineno=7)
          40	BINARY_SUBTRACT(arg=None, lineno=7)
          42	LOAD_FAST(arg=2, lineno=7)
          44	LO

DEBUG:numba.core.byteflow:dispatch pc=74, inst=BINARY_MULTIPLY(arg=None, lineno=8)
DEBUG:numba.core.byteflow:stack ['$60binary_multiply.27', '$66binary_subscr.30', '$72binary_subscr.33']
DEBUG:numba.core.byteflow:dispatch pc=76, inst=BINARY_SUBTRACT(arg=None, lineno=8)
DEBUG:numba.core.byteflow:stack ['$60binary_multiply.27', '$74binary_multiply.34']
DEBUG:numba.core.byteflow:dispatch pc=78, inst=LOAD_FAST(arg=2, lineno=8)
DEBUG:numba.core.byteflow:stack ['$76binary_subtract.35']
DEBUG:numba.core.byteflow:dispatch pc=80, inst=LOAD_CONST(arg=2, lineno=8)
DEBUG:numba.core.byteflow:stack ['$76binary_subtract.35', '$c78.36']
DEBUG:numba.core.byteflow:dispatch pc=82, inst=STORE_SUBSCR(arg=None, lineno=8)
DEBUG:numba.core.byteflow:stack ['$76binary_subtract.35', '$c78.36', '$const80.37']
DEBUG:numba.core.byteflow:dispatch pc=84, inst=LOAD_FAST(arg=0, lineno=9)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=86, inst=LOAD_CONST(arg=4, lineno=9)
DEBUG:numba.core.bytefl

DEBUG:numba.core.ssa:==== SSA block analysis pass on 0
DEBUG:numba.core.ssa:Running <numba.core.ssa._GatherDefsHandler object at 0x2b760d0ee910>
DEBUG:numba.core.ssa:on stmt: a = arg(0, name=a)
DEBUG:numba.core.ssa:on stmt: b = arg(1, name=b)
DEBUG:numba.core.ssa:on stmt: $2load_global.0 = global(np: <module 'numpy' from '/home/zhangi/miniconda3/envs/perses-rbd-ace2-direct/lib/python3.8/site-packages/numpy/__init__.py'>)
DEBUG:numba.core.ssa:on stmt: $4load_method.1 = getattr(value=$2load_global.0, attr=zeros)
DEBUG:numba.core.ssa:on stmt: $const6.2 = const(int, 3)
DEBUG:numba.core.ssa:on stmt: c = call $4load_method.1($const6.2, func=$4load_method.1, args=[Var($const6.2, coordinate_numba.py:6)], kws=(), vararg=None)
DEBUG:numba.core.ssa:on stmt: $const14.5 = const(int, 1)
DEBUG:numba.core.ssa:on stmt: $16binary_subscr.6 = static_getitem(value=a, index=1, index_var=$const14.5, fn=<built-in function getitem>)
DEBUG:numba.core.ssa:on stmt: $const20.8 = const(int, 2)
DEBUG:numba.core.ssa:

DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=0 nstack_initial=0)])
DEBUG:numba.core.byteflow:stack: []
DEBUG:numba.core.byteflow:dispatch pc=0, inst=NOP(arg=None, lineno=14)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=2, inst=LOAD_GLOBAL(arg=0, lineno=14)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=4, inst=LOAD_METHOD(arg=1, lineno=14)
DEBUG:numba.core.byteflow:stack ['$2load_global.0']
DEBUG:numba.core.byteflow:dispatch pc=6, inst=LOAD_FAST(arg=0, lineno=14)
DEBUG:numba.core.byteflow:stack ['$4load_method.1']
DEBUG:numba.core.byteflow:dispatch pc=8, inst=LOAD_FAST(arg=0, lineno=14)
DEBUG:numba.core.byteflow:stack ['$4load_method.1', '$a6.2']
DEBUG:numba.core.byteflow:dispatch pc=10, inst=CALL_METHOD(arg=2, lineno=14)
DEBUG:numba.core.byteflow:stack ['$4load_method.1', '$a6.2', '$a8.3']
DEBUG:numba.core.byteflow:dispatch pc=12, inst=STORE_FAST(arg=1, lineno=14)
DEBUG:numba.core.byteflow:stack ['$10call_method.4']
D

DEBUG:numba.core.byteflow:stack ['$b10.4']
DEBUG:numba.core.byteflow:dispatch pc=14, inst=UNPACK_SEQUENCE(arg=1, lineno=509)
DEBUG:numba.core.byteflow:stack ['$12load_attr.5']
DEBUG:numba.core.byteflow:dispatch pc=16, inst=STORE_FAST(arg=3, lineno=509)
DEBUG:numba.core.byteflow:stack ['$14unpack_sequence.6']
DEBUG:numba.core.byteflow:dispatch pc=18, inst=LOAD_FAST(arg=2, lineno=510)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=20, inst=LOAD_FAST(arg=3, lineno=510)
DEBUG:numba.core.byteflow:stack ['$m18.8']
DEBUG:numba.core.byteflow:dispatch pc=22, inst=COMPARE_OP(arg=3, lineno=510)
DEBUG:numba.core.byteflow:stack ['$m18.8', '$n20.9']
DEBUG:numba.core.byteflow:dispatch pc=24, inst=POP_JUMP_IF_FALSE(arg=34, lineno=510)
DEBUG:numba.core.byteflow:stack ['$22compare_op.10']
DEBUG:numba.core.byteflow:end state. edges=[Edge(pc=26, stack=(), blockstack=(), npush=0), Edge(pc=34, stack=(), blockstack=(), npush=0)]
DEBUG:numba.core.byteflow:pending: deque([State(pc_ini

...

DEBUG:numba.core.ssa:==== SSA block rewrite pass on 0
DEBUG:numba.core.ssa:Running <numba.core.ssa._FixSSAVars object at 0x2b760d63f9a0>
DEBUG:numba.core.ssa:on stmt: atom_position = arg(0, name=atom_position)
DEBUG:numba.core.ssa:on stmt: bond_position = arg(1, name=bond_position)
DEBUG:numba.core.ssa:on stmt: angle_position = arg(2, name=angle_position)
DEBUG:numba.core.ssa:on stmt: torsion_position = arg(3, name=torsion_position)
DEBUG:numba.core.ssa:on stmt: a = atom_position - bond_position
DEBUG:numba.core.ssa:on stmt: b = angle_position - bond_position
DEBUG:numba.core.ssa:on stmt: c = angle_position - torsion_position
DEBUG:numba.core.ssa:on stmt: $28load_global.10 = global(_norm: CPUDispatcher(<function _norm at 0x2b760cc223a0>))
DEBUG:numba.core.ssa:on stmt: $32call_function.12 = call $28load_global.10(a, func=$28load_global.10, args=[Var(a, coordinate_numba.py:106)], kws=(), vararg=None)
DEBUG:numba.core.ssa:on stmt: a_u = a / $32call_function.12
DEBUG:numba.core.ssa:on stmt

DEBUG:numba.core.ssa:on stmt: cos_phi.2 = const(float, 1.0)
DEBUG:numba.core.ssa:on stmt: jump 192
DEBUG:numba.core.ssa:==== SSA block rewrite pass on 192
DEBUG:numba.core.ssa:Running <numba.core.ssa._FixSSAVars object at 0x2b760d63f9a0>
DEBUG:numba.core.ssa:on stmt: $192load_global.0 = global(np: <module 'numpy' from '/home/zhangi/miniconda3/envs/perses-rbd-ace2-direct/lib/python3.8/site-packages/numpy/__init__.py'>)
DEBUG:numba.core.ssa:on stmt: $194load_method.1 = getattr(value=$192load_global.0, attr=arccos)
DEBUG:numba.core.ssa:on stmt: phi = call $194load_method.1(cos_phi, func=$194load_method.1, args=[Var(cos_phi, coordinate_numba.py:129)], kws=(), vararg=None)
DEBUG:numba.core.ssa:find_def var='cos_phi' stmt=phi = call $194load_method.1(cos_phi, func=$194load_method.1, args=[Var(cos_phi, coordinate_numba.py:129)], kws=(), vararg=None)
DEBUG:numba.core.ssa:find_def_from_top label 192
DEBUG:numba.core.ssa:insert phi node cos_phi.3 = phi(incoming_values=[], incoming_blocks=[]) at 

DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=0 nstack_initial=0)])
DEBUG:numba.core.byteflow:stack: []
DEBUG:numba.core.byteflow:dispatch pc=0, inst=NOP(arg=None, lineno=106)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=2, inst=LOAD_FAST(arg=0, lineno=106)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=4, inst=LOAD_FAST(arg=1, lineno=106)
DEBUG:numba.core.byteflow:stack ['$angle_position_12.0']
DEBUG:numba.core.byteflow:dispatch pc=6, inst=BINARY_SUBTRACT(arg=None, lineno=1)
DEBUG:numba.core.byteflow:stack ['$angle_position_12.0', '$bond_position_24.1']
DEBUG:numba.core.byteflow:dispatch pc=8, inst=RETURN_VALUE(arg=None, lineno=1)
DEBUG:numba.core.byteflow:stack ['$6binary_subtract.2']
DEBUG:numba.core.byteflow:end state. edges=[]
DEBUG:numba.core.byteflow:-------------------------Prune PHIs-------------------------
DEBUG:numba.core.byteflow:Used_phis: defaultdict(<class 'set'>, {State(pc_initial=0 nstack_initial=0): s

DEBUG:numba.core.byteflow:keep phismap: {}
DEBUG:numba.core.byteflow:new_out: defaultdict(<class 'dict'>, {})
DEBUG:numba.core.byteflow:----------------------DONE Prune PHIs-----------------------
DEBUG:numba.core.byteflow:block_infos State(pc_initial=0 nstack_initial=0):
AdaptBlockInfo(insts=((0, {}), (2, {'res': '$a_12.0'}), (4, {'res': '$_32call_function_12_14.1'}), (6, {'lhs': '$a_12.0', 'rhs': '$_32call_function_12_14.1', 'res': '$6binary_true_divide.2'}), (8, {'retval': '$6binary_true_divide.2', 'castval': '$8return_value.3'})), outgoing_phis={}, blockstack=(), active_try_block=None, outgoing_edgepushed={})
DEBUG:numba.core.interpreter:label 0:
    _32call_function_12_1 = arg(0, name=_32call_function_12_1) ['_32call_function_12_1']
    a_1 = arg(1, name=a_1)                   ['a_1']
    $6binary_true_divide.2 = a_1 / _32call_function_12_1 ['$6binary_true_divide.2', '_32call_function_12_1', 'a_1']
    $8return_value.3 = cast(value=$6binary_true_divide.2) ['$6binary_true_divide.2'

DEBUG:numba.core.ssa:on stmt: c_1 = arg(1, name=c_1)
DEBUG:numba.core.ssa:on stmt: $6binary_true_divide.2 = c_1 / _56call_function_22_1
DEBUG:numba.core.ssa:on stmt: $8return_value.3 = cast(value=$6binary_true_divide.2)
DEBUG:numba.core.ssa:on stmt: return $8return_value.3
DEBUG:numba.core.ssa:defs defaultdict(<class 'list'>,
            {'$6binary_true_divide.2': [<numba.core.ir.Assign object at 0x2b760d935460>],
             '$8return_value.3': [<numba.core.ir.Assign object at 0x2b760d935e80>],
             '_56call_function_22_1': [<numba.core.ir.Assign object at 0x2b760d4b7a00>],
             'c_1': [<numba.core.ir.Assign object at 0x2b760d4b7b20>]})
DEBUG:numba.core.ssa:SSA violators set()
INFO:rdkit:Enabling RDKit 2021.03.4 jupyter extensions


In [2]:

# Get alanine dipeptide in vacuum test system
ala, system_generator = generate_atp()
    
    
# ALA -> ILE
topology_proposal, new_positions, logp_proposal, logp_reverse = generate_dipeptide_top_pos_sys(ala.topology, 
                                                                                                   'ILE', 
                                                                                                   ala.system, 
                                                                                                   ala.positions, 
                                                                                                   system_generator, 
                                                                                                   conduct_geometry_prop=True)
# reference_map = [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('C', 'C'), ('O', 'O'), ('CB', 'CB'), ('HB', 'HB')]
# check_atom_map(topology_proposal, reference_map)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:local_atom_map: {6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 14, 12: 15}
INFO:proposal_generator:the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('CB', 'CB'), ('C', 'C'), ('O', 'O')]
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 12
INFO:geometry:Atom index proposal order is [18, 14, 21, 13, 19, 20, 23, 17, 22, 15, 16, 24]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INF

making topology proposal
generating geometry engine
making geometry proposal from ALA to ILE


INFO:geometry:log probability choice of torsions and atom order: -25.656082814775196
INFO:geometry:creating platform, integrators, and contexts; setting growth parameter


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced potential before atom placement: 16.81406823661637


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 8.816525446865816
INFO:geometry:final reduced energy 25.630593424131863
INFO:geometry:sum of energies: 25.63059368348219
INFO:geometry:magnitude of difference in the energies: 2.5935032432755634e-07
INFO:geometry:Final logp_proposal: -26900.657005873363


added energy components: [('CustomBondForce', 0.0), ('CustomAngleForce', 0.00017810081275281765), ('CustomTorsionForce', 0.004336815681512571), ('CustomBondForce', 8.812010530371548)]


In [3]:
app.PDBFile.writeFile(topology_proposal.new_topology, new_positions, open("ile.pdb", "w"))